In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
import seaborn as sns
from statsmodels.tsa.stattools import grangercausalitytests, ccf
import numpy as np
import plotly.express as px
from datetime import datetime
import geopandas as gpd
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

sys.path.append(os.path.abspath("../scripts"))  # Adjust the path
os.chdir(os.path.dirname(os.path.abspath("__file__")))  # Set working directory

from utils import *

from utils import plot_outages_on_map_us

## 1.  Environment Setup

✔ Ensure all dependencies are installed using requirements.txt

In [2]:
pip install -r "../dynamic_rhythms/requirements.txt"

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../dynamic_rhythms/requirements.txt'
Note: you may need to restart the kernel to use updated packages.


## 2. Data Loading & Exploration

### 2.1 Load Power Outage Data
- Read .csv files in ../dynamic_rhythm_env/eaglei_data/
- Inspect columns, data types, and missing values
- Identify key variables like state, run_start_time, customers_out


In [3]:
# fileURL = "https://d396qusza40orc.cloudfront.net/repdata%2Fdata%2FStormData.csv.bz2"
# dest = "repdata_data_StormData.csv.bz2"
# if(!file.exists(dest))
#     download.file(fileURL,dest)
# storm = read.csv(dest)


# setwd(".")
# download.file("https://d396qusza40orc.cloudfront.net/repdata%2Fdata%2FStormData.csv.bz2", destfile = "./repdata-data-StormData.csv.bz2", method = "curl")
# storm <- read.csv("repdata-data-StormData.csv.bz2", header = TRUE)

In [4]:
# import kagglehub

# # Download latest  
# path = kagglehub.dataset_download("sobhanmoosavi/us-weather-events")

# print("Path to dataset files:", path)

In [5]:
# Define dataset directories
outages_dir = "../dynamic_rhythm_train_data/eaglei_data/"
storms_dir = "../dynamic_rhythm_train_data/NOAA_StormEvents/"

# List files
outage_files = os.listdir(outages_dir)
storm_files = os.listdir(storms_dir)

print("Power Outage Files:", outage_files)
print("Storm Event Files:", storm_files)



Power Outage Files: ['DQI_processing.R', 'DQI.csv', 'eaglei_outages_2019.csv', 'eaglei_outages_2018.csv', 'eaglei_outages_2023.csv', 'MCC.csv', 'eaglei_outages_2022.csv', 'eaglei_outages_2020.csv', 'eaglei_outages_2021.csv', 'coverage_history.csv', 'eaglei_outages_2016.csv', 'eaglei_outages_2017.csv', 'eaglei_outages_2015.csv', 'eaglei_outages_2014.csv', 'Uri_Map.R']
Storm Event Files: ['StormEvents_details-ftp_v1.0_d2014_c20231116.csv', 'StormEvents_details-ftp_v1.0_d2019_c20240117.csv', 'StormEvents_details-ftp_v1.0_d2021_c20240716.csv', 'StormEvents_details-ftp_v1.0_d2018_c20240716.csv', 'StormEvents_details-ftp_v1.0_d2024_c20241216.csv', 'StormEvents_details-ftp_v1.0_d2023_c20241216.csv', 'StormEvents_details-ftp_v1.0_d2016_c20220719.csv', 'StormEvents_details-ftp_v1.0_d2017_c20230317.csv', 'StormEvents_details-ftp_v1.0_d2015_c20240716.csv', 'StormEvents_details-ftp_v1.0_d2022_c20241121.csv', 'StormEvents_details-ftp_v1.0_d2020_c20240620.csv', 'StormEvents_2014_2024.csv']


In [6]:
print("\n Power Outage Files:")
for file in outage_files:
    print(f"   - {file} ({file.split('.')[-1]})")  # Show file extensions

print("\n Storm Event Files:")
for file in storm_files:
    print(f"   - {file} ({file.split('.')[-1]})")

# Function to preview CSV files
def preview_csv(file_path, num_rows=5):
    """Load and preview a CSV file."""
    try:
        df = pd.read_csv(file_path, nrows=num_rows)  
        print(f"\n🔍 Preview of {os.path.basename(file_path)}:")
        print(df.head())  
        print("\n📊 Column Info:")
        print(df.info())  
        print("\n❗ Missing Values:")
        print(df.isnull().sum())  
    except Exception as e:
        print(f" Error loading {file_path}: {e}")

# Preview power outage CSV datasets
print("\n **Power Outage Data Preview**")
for file in outage_files:
    if file.endswith(".csv"):
        preview_csv(os.path.join(outages_dir, file))

# Preview storm event CSV datasets
print("\n **Storm Event Data Preview**")
for file in storm_files:
    if file.endswith(".csv"):
        preview_csv(os.path.join(storms_dir, file))

# Highlight non-CSV files for manual review
non_csv_files = [f for f in outage_files + storm_files if not f.endswith(".csv")]
if non_csv_files:
    print("\n **Non-CSV Files Found:**")
    for file in non_csv_files:
        print(f"   - {file}")
    print(" Please check these manually—some may contain useful information.")



 Power Outage Files:
   - DQI_processing.R (R)
   - DQI.csv (csv)
   - eaglei_outages_2019.csv (csv)
   - eaglei_outages_2018.csv (csv)
   - eaglei_outages_2023.csv (csv)
   - MCC.csv (csv)
   - eaglei_outages_2022.csv (csv)
   - eaglei_outages_2020.csv (csv)
   - eaglei_outages_2021.csv (csv)
   - coverage_history.csv (csv)
   - eaglei_outages_2016.csv (csv)
   - eaglei_outages_2017.csv (csv)
   - eaglei_outages_2015.csv (csv)
   - eaglei_outages_2014.csv (csv)
   - Uri_Map.R (R)

 Storm Event Files:
   - StormEvents_details-ftp_v1.0_d2014_c20231116.csv (csv)
   - StormEvents_details-ftp_v1.0_d2019_c20240117.csv (csv)
   - StormEvents_details-ftp_v1.0_d2021_c20240716.csv (csv)
   - StormEvents_details-ftp_v1.0_d2018_c20240716.csv (csv)
   - StormEvents_details-ftp_v1.0_d2024_c20241216.csv (csv)
   - StormEvents_details-ftp_v1.0_d2023_c20241216.csv (csv)
   - StormEvents_details-ftp_v1.0_d2016_c20220719.csv (csv)
   - StormEvents_details-ftp_v1.0_d2017_c20230317.csv (csv)
   - StormEv


---

**Observations:**
1. **No Major Missing Data Issues**  
   - Most datasets have complete records.  
   - **Exception:** `eaglei_outages_2020.csv` has a missing value in `customers_out`.

2. **Date Format Consistency Check**  
   - The `run_start_time` column in outage datasets is stored as an **object (string)**.  
   - We need to **convert it to datetime** for time-based analysis.

3. **County-Level Consistency**  
   - Some datasets use `fips_code`, while others (like `MCC.csv`) use `County_FIPS`.  
   - We should ensure **consistent naming and datatype alignment** before merging.

4. **Power Outage Data Granularity**  
   - Outage datasets (`eaglei_outages_YYYY.csv`) contain **county-level** power outage records.  
   - `MCC.csv` provides **total customer counts per county**, which can be used for **normalization** (outages per 1000 customers).  
   - `DQI.csv` and `coverage_history.csv` contain **data quality indicators and coverage trends**, useful for filtering.

---

**Next Steps:**
**Step 1:** Convert `run_start_time` to datetime.  
**Step 2:** Standardize column names (e.g., `County_FIPS` → `fips_code`).  
**Step 3:** Handle missing values (`customers_out` in 2020).  
**Step 4:** Merge datasets (Outages + MCC + Coverage + DQI).  
**Step 5:** Compute new features, e.g., **outage rate per 1000 customers**.

---
---
---


The **big picture**:

---

**Goal of the Constellation EAGLE-I Power Outage Challenge**:

**Overall Objective:**  
> Use historical outage data (EAGLE-I) + extreme weather events (NOAA) to model, analyze, and forecast patterns in *power outages*.

---

**More specifically, the challenge asks to:**

1. **Understand** the relationships between weather events and power outages.  
   (e.g., *Does ice storm frequency explain outage spikes?*)

2. **Build predictive models** for outages:
   - Given a set of upcoming weather conditions (storm types, frequency),
   - **Predict** the expected number of customers who might lose power.

3. **Create informative visualizations** and **report insights**:
   - Show how outages vary across **states**, **time**, **event types**.
   - Show correlations between specific **storm types** and **outages**.

4. **Innovate** with forecasting:
   - Can you forecast future outage risks?
   - Can you identify **critical areas** where infrastructure is more vulnerable?

---

**So practically:**
- combine **outage time series** + **storm events time series**.
- **model** outages using weather variables.
- **analyze**, **forecast**, and **explain** the patterns.

---

**In short:**

- **Not just** merging data.
- **Not just** visualizing.
- **Ultimately** → *forecast* or *predict* how bad outages could get under different weather conditions.

---


---

**"Official" Datasets (provided or expected):**
| Data | Purpose |
|:----|:--------|
| EAGLE-I Outages (2014–2023) | Actual counts of customers out, by time and location. |
| NOAA Storm Events Database (2014–2023) | Detailed records of extreme weather events: type, time, severity. |

---

**"Extra" Datasets We *Can* Use:**

| Data | Why it might help |
|:----|:-------------------|
| **Weather** (temperature, wind speed, snow depth, rainfall, etc.) | Outages often happen because of *severe* versions of normal weather. |
| **Population density** | More populated areas might report more customers out even for similar storms. |
| **Infrastructure resilience data** (if available) | Areas with underground cables might suffer fewer outages. |
| **Previous large blackout datasets** | To find patterns (e.g., cascading outages after storms). |
| **Climate trends** (e.g., NOAA climate normals) | To detect if outages are increasing due to more frequent extreme events. |
| **Energy infrastructure locations** (substations, grid maps) | To check if outages cluster near vulnerable assets. |

---

**Why add extra data?:**

- Improve **feature engineering** → better predictive models.
- Explain *why* certain areas have more severe outages (not just storms!).
- Build a more **resilient forecasting** system that considers multiple stressors.

---

- We **don't need extra data** to complete the challenge.
- **But** bringing it in **could make your solution much stronger**, smarter, and more impressive to the judges.

---


---

**Must Do**

| Section | Key Actions We Need to Take |
|:---|:---|
| **1. Objective & Approach** | Predict the **occurrence**, **lead time**, **severity**, and/or **duration** of **power outages** from storms (or rare weather). We must **combine datasets** and justify your design choices. |
| **2. Data & Feature Engineering** | Use the provided **storm** and **outage** datasets, plus optionally **external public data** (weather, population, infrastructure). We must **align** them carefully in **time** and **space** (e.g., county and date). |
| **3. Modeling & Prediction** | Handle **rare events** carefully (imbalanced data). Predict ahead in **lead time** and **severity** (not just occurrence). **Choose appropriate ML models** (classic ML, time series models, or even deep learning if you want). |
| **4. Metrics & Evaluation** | **Design your own metrics**! Suggested types: F1 score for rare events, lead-time error, severity prediction accuracy, location-based error. Explain and justify your evaluation methods. |
| **5. Submission** | Submit **code + docs**: preprocessing, feature engineering, modeling, and evaluation all clearly shown. Make results **reproducible**. Include **insights** (which features mattered most). Creativity is encouraged! |

---

**In simple terms:**
Predict **if** an outage will happen,  
Predict **when** (lead time),  
Predict **how bad** (customers out, duration),  
Combine multiple datasets creatively and carefully,  
Choose your own models, metrics, and validation strategies,  
Show clear analysis, visualizations, and insights.

---

**Now, regarding your earlier question about **extra data**:  **
→ **YES**, you can (and *should*) use additional public datasets — weather, population, critical infrastructure, etc.  
→ The goal is to create a model that feels as **real-world predictive** as possible.

---

**Immediate Next Steps I Recommend**

| Step | What to Do |
|:---|:---|
| 1 | Finalize core datasets: Storm Events, Outages, and optionally Population or Weather Data. |
| 2 | Build **state/county/time-aligned** tables combining outage counts and storm features. |
| 3 | Start **feature engineering**: like event type, magnitude, lagged weather, county population normalization. |
| 4 | Split data **time-wise** (e.g., train on 2014–2020, test on 2021–2023). |
| 5 | Build **first simple model** (XGBoost classifier for outage yes/no, for example). |
| 6 | Design **custom metrics**: F1 for rare events, lead time error, severity prediction error. |

---



---
---
---

**Power Outage Forecasting Pipeline**

**1. **Data Collection****

- **Load Provided Data**:
  - Outages: (`outages14_23_texas_df`)
  - Storm Events: (`storm_events_df`)
- **Optional External Data**:
  - Population Data (by county)
  - ERA5 Weather Data (temperature, wind, precipitation)

---

**2. **Preprocessing & Alignment****

- **Fix Timestamps**: Parse all dates correctly.
- **Fix Locations**: Standardize counties (maybe use FIPS codes).
- **Filter**: Focus on Texas or selected states.
- **Create Event Windows**: Match storms to outage periods.
- **Aggregate**: 
  - Outages → county-day level (`total_customers_out`)
  - Storms → county-day level (summarized storm features)

---

**3. **Feature Engineering****

- **Storm Features**:
  - Event type (hurricane, thunderstorm, etc.)
  - Event severity (wind speed, property damage, etc.)
- **Outage Features**:
  - Rolling customers out (lags, moving averages)
- **Time Features**:
  - Day of week, seasonality (e.g., hurricane season)
- **Population or Infra Features** (if external data):
  - Normalize outages per capita
  - Critical infrastructure density
- **Lagged/Lead Features**:
  - Weather 1–3 days before outage

---

**4. **Label Creation****

- **Classification Target**:
  - 1 = Major outage (e.g., >500 customers affected)
  - 0 = No major outage
- **Severity Target** (optional regression):
  - Predict number of customers affected
- **Lead Time Target** (optional):
  - How many hours/days before the outage?

---

**5. **Modeling****

- **Baseline**: Logistic Regression, Random Forest, XGBoost
- **Advanced**: 
  - Time series models (Prophet, LSTM)
  - Transformer-based models if ambitious
- **Model Outputs**:
  - Outage occurrence (yes/no)
  - Outage severity (customers_out, duration)

---

**6. **Evaluation****

- **Classification Metrics**:
  - F1 Score
  - Precision-Recall AUC
- **Lead Time Metrics**:
  - Mean absolute lead time error
- **Severity Prediction Metrics**:
  - MAE / RMSE on customers affected
- **Spatial Metrics**:
  - County-level location accuracy

---

**7. **Insights & Visualization****

- **Feature Importances** (from tree models)
- **Partial Dependence Plots** (PDP)
- **Error Analysis**:
  - Which counties or seasons are hardest to predict?
- **Maps**:
  - Outage heatmaps vs prediction maps

---

**8. **Submission****

- Organized, reproducible notebooks/scripts
- Requirements.txt or environment.yml
- Summary of external datasets used
- Documentation for rerunning everything

---

**Visual Sketch of the Pipeline**

```plaintext
📂 Data Loading
    ↓
🔧 Preprocessing & Alignment
    ↓
🎛️ Feature Engineering
    ↓
🏷️ Labeling
    ↓
🧠 Modeling
    ↓
📈 Evaluation
    ↓
📊 Insights & Visualization
    ↓
📝 Submission Package
```

---

**Minimal MVP version:**
- Use only outages + storm events.
- Make a binary classifier (outage yes/no next day).
- Use Random Forest or XGBoost baseline.
- Design a simple F1 + lead-time MAE metric.
- Then later add external data if time permits.

---


The challenge is about **predicting outages** caused by **extreme weather** — *where*, *when*, *how severe*.  
So, the most important features will be those that capture:

---

1. **Weather severity and type** (→ *What happened?*)

- `event_type` (storm, tornado, flood, etc.)
- `event_magnitude` (e.g., wind speed, rainfall amount, hail size)
- `event_duration` (length of the storm)
- `event_start_time`, `event_end_time`
- `storm category` (for hurricanes, if available)

---
2. **Location and exposure** (→ *Where and how vulnerable?*)

- `county`, `state`, `fips_code`
- **Population density** (external data)  
  → More people = more customers affected
- **Urban vs. Rural indicator** (external or derived)  
  → Rural areas might recover differently than urban
- **Infrastructure vulnerability** (optional external data)  
  → e.g., number of substations, grid strength if available

---
3. **Temporal context** (→ *When did it happen?*)

- **Seasonality features**:
  - `month`, `day of week`, `hour of day`
  - (`storm_season` indicator: e.g., hurricane season June-Nov)
- **Lead time / time gaps** between storms and outages

---
4. **Historical storm and outage frequency** (→ *Is this place usually hit?*)

- **Storm frequency** by county over past X months
- **Outage frequency** by county over past X months
- (rolling averages, lagged counts)

---
5. **Magnitude of previous outages** (→ *How bad was it before?*)

- `avg_customers_out_last_month`
- `max_customers_out_last_year`
- `average outage duration in county`

---
6. **Alignment features** (critical for linking storms to outages)

- `storm_to_outage_time_gap`
- `storm_event_matches_outage` (binary: 1 if storm caused an outage, 0 otherwise — for training)

---

**Most important categories**:
- **Storm strength/severity**
- **Location risk (population, infrastructure)**
- **Timing (seasonality, lead time)**
- **Historical vulnerability (previous storms, outages)**

---



---

📋 Draft Feature Table:  
*(includes what we already have from the provided datasets)*

| Feature Name | Source Columns (or Derived) | Notes | Do we already have it? |
|:-------------|:----------------------------|:------|:-----------------------|
| `event_type` | storm events `event_type` | What kind of storm (e.g., thunderstorm, hurricane) | ✅ |
| `event_magnitude` | storm events (depends: wind speed, hail size, etc.) | Severity; may need parsing different columns | ⚠️ *depends on subtype* |
| `event_start_time` | storm events `event_begin_time` | When the storm started | ✅ |
| `event_end_time` | storm events `event_end_time` | When the storm ended | ✅ |
| `county` | storm events `cz_name` and outages `county` | Match both datasets spatially | ✅ |
| `state` | storm events `state` and outages `state` | Should match too | ✅ |
| `fips_code` | outages `fips_code` | County-level ID for matching | ✅ |
| `customers_out` | outages `customers_out` | Target for severity prediction | ✅ |
| `outage_time` | outages `run_start_time` | When outages were recorded | ✅ |
| `event_duration` | Derived (`event_end_time` - `event_start_time`) | Duration of storm | ➡️ Easy to compute |
| `outage_lead_time` | Derived (`outage_time` - `event_end_time`) | How soon after storm the outage happened | ➡️ Easy to compute |
| `storm_season` | Derived from `event_start_time` | E.g., flag June–Nov as hurricane season | ➡️ Easy to compute |
| `month` | Derived from `event_start_time` | Month of year (seasonality) | ➡️ Easy to compute |
| `day_of_week` | Derived from `event_start_time` | Day of week (temporal patterns) | ➡️ Easy to compute |
| `storm_count_past_X_days` | Rolling count from storm data | How stormy was it recently? | ❌ *(needs aggregation)* |
| `outage_count_past_X_days` | Rolling count from outage data | Outage-prone counties | ❌ *(needs aggregation)* |
| `population_density` | External (e.g., census) | More people = bigger impact | ❌ *(external lookup)* |
| `urban_rural_indicator` | External (e.g., USDA) | City vs rural areas | ❌ *(external lookup)* |
| `infrastructure_risk` | External (optional) | Power grid vulnerability | ❌ *(if available)* |

---

✅ = already available directly  
➡️ = easy to engineer from available data  
❌ = would require external data or more work  
⚠️ = depends: need to decide which storm subtype columns to use for "magnitude"

---

---

In [7]:
def load_outage_selected_cols(outages_dir, columns=['state', 'customers_out', 'run_start_time']):
    """
    Efficiently loads selected columns from all outage CSV files in a directory.

    Parameters:
    - outages_dir (str): Path to directory with outage CSVs.
    - columns (list): List of columns to load from each file.

    Returns:
    - pd.DataFrame: Concatenated DataFrame with selected columns across all files.
    """

    outage_files = sorted([
        f for f in os.listdir(outages_dir)
        if f.startswith("eaglei_outages_") and f.endswith(".csv")
    ])

    df_list = []
    for f in tqdm(outage_files, desc="Loading outage files", unit="file"):
        try:
            df = pd.read_csv(os.path.join(outages_dir, f), usecols=columns)
            df_list.append(df)
        except Exception as e:
            print(f"Skipping {f}: {e}")

    if df_list:
        outages_df = pd.concat(df_list, ignore_index=True)
        return outages_df
    else:
        print("No outage data loaded.")
        return pd.DataFrame()

outages_selected_df = load_outage_selected_cols(outages_dir)

outages_selected_df


Loading outage files: 100%|██████████| 10/10 [00:54<00:00,  5.41s/file]


,state,customers_out,run_start_time
0,Alabama,12.0,2014-11-01 04:00:00
1,Alabama,7.0,2014-11-01 04:00:00
2,Alabama,1.0,2014-11-01 04:00:00
3,Alabama,31.0,2014-11-01 04:00:00
4,Arizona,1.0,2014-11-01 04:00:00
...,...,...,...
191133063,Wisconsin,0.0,2023-12-31 23:45:00
191133064,Wisconsin,1.0,2023-12-31 23:45:00
191133065,Wisconsin,0.0,2023-12-31 23:45:00
191133066,Wisconsin,0.0,2023-12-31 23:45:00


In [8]:
# outages_selected_df.info()

# Normalize inconsistent state names
normalize_state_names = {
    'US Virgin Islands': 'United States Virgin Islands',
    'U.S. Virgin Islands': 'United States Virgin Islands',
    'Virgin Islands': 'United States Virgin Islands',
    'District Of Columbia': 'District of Columbia',
}

outages_selected_df['state'] = outages_selected_df['state'].replace(normalize_state_names)


In [9]:
# Now query top states
state_outages_agg = (
    outages_selected_df.groupby('state')
    .agg(
        total_outages=('state', 'count'),
        total_customers_out=('customers_out', 'sum'),
        avg_customers_out=('customers_out', 'mean')
    )
    .sort_values(by='total_customers_out', ascending=False)
)

top_outages_states_df = state_outages_agg.head(5)
least_outages_states_df = state_outages_agg.tail(5)

print("List of Top 5 states by total customers out:", top_outages_states_df.index.tolist())
display(top_outages_states_df)

print("List Least 5 states by total customers out:", least_outages_states_df.index.tolist())
display(least_outages_states_df)


List of Top 5 states by total customers out: ['Florida', 'California', 'Texas', 'Louisiana', 'Michigan']


,total_outages,total_customers_out,avg_customers_out
state,,,
Florida,9696235,4.269705e+09,452.182221
California,7310527,2.880102e+09,405.826175
Texas,16413490,2.745459e+09,172.549290
Louisiana,6619865,2.198361e+09,338.634795
Michigan,6894865,2.160075e+09,332.079810


List Least 5 states by total customers out: ['Montana', 'South Dakota', 'District of Columbia', 'North Dakota', 'Wyoming']


,total_outages,total_customers_out,avg_customers_out
state,,,
Montana,749199,45331082.0,68.904579
South Dakota,1586739,42664671.0,31.050350
District of Columbia,202886,27338954.0,135.385591
North Dakota,874415,23266926.0,39.501348
Wyoming,650641,17728228.0,29.647552


- The entire outages dataset was too large for analysis and modelling (7GB), so it's best to take portions
capturing top and least impacted which are more insightful

- we can then go back and filter for these group with all columns (fips, county, etc) for future granularity. These reduces the size of dataset to work with


---


**Top 5 States by Total Customers Out**
These are high-priority candidates for predictive modeling, especially if we're focused on **impact severity**:

1. **Florida** – Highest overall impact (4.27B customers out), very high average per outage.
2. **California** – Frequent outages and high total impact.
3. **Texas** – Most frequent outages overall, but lower average per event.
4. **Louisiana** – Fewer events than CA/TX but higher average impact per outage.
5. **Michigan** – Similar to Louisiana in impact profile.

These states provide a **diverse mix**: hurricane-prone (FL, LA), wildfire-prone (CA), storm/cold-prone (MI), and grid-stress-prone (TX). This variation is useful for training more generalizable models.

---

**Bottom 5 States**
These are low-impact zones (by total `customers_out`), and generally **not ideal** for early modeling due to sparse or low-severity data:

**['Montana', 'South Dakota', 'District of Columbia', 'North Dakota', 'Wyoming']**

These states may serve better as controls or for testing model generalizability later.

---

**Follow up:**
Now that you have the top 5/ least 5 states:
- Go back and reload full outage data for **top5/least5**, including all columns.
- Then, filter corresponding storm event records for just those states.


---

**Why Use Top 10 Instead of Just 5?**
- **Model robustness**: Broader geographic and climatic diversity leads to better generalization.
- **Extreme events**: Some states may be low on *total* outages but high on *rare/extreme* events — these patterns only emerge with a larger sample.
- **Comparative evaluation**: We’ll be able to better test whether models trained on top 5 generalize well to states ranked 6–10.
- **Future flexibility**: If some states become unusable due to data issues or require exclusion, we already have backups in the top 10.

---

In [10]:
least_outages_states_df.index.tolist()

['Montana', 'South Dakota', 'District of Columbia', 'North Dakota', 'Wyoming']

In [11]:
top_least_10_outages_states = top_outages_states_df.index.tolist() + least_outages_states_df.index.tolist()
print(top_least_10_outages_states)

['Florida', 'California', 'Texas', 'Louisiana', 'Michigan', 'Montana', 'South Dakota', 'District of Columbia', 'North Dakota', 'Wyoming']


In [12]:

def prepare_outages_dataset(states, outages_dir):
    """
    Reads and filters outage CSVs for one or more states.
    
    Parameters:
    - states (str or list of str): State name(s) to filter by.
    - outages_dir (str): Path to directory containing outage CSVs.

    Returns:
    - pd.DataFrame: Combined outages DataFrame filtered by state(s).
    """
    if isinstance(states, str):
        states = [states]
        
    outage_files = sorted([
        f for f in os.listdir(outages_dir) 
        if f.startswith("eaglei_outages_") and f.endswith(".csv")
    ])

    filtered_outages = []

    for f in tqdm(outage_files, desc=f"Processing outages for {states}", unit="file"):
        file_path = os.path.join(outages_dir, f)
        try:
            df = pd.read_csv(file_path, parse_dates=['run_start_time'])
            df_state = df[df['state'].isin(states)]
            if not df_state.empty:
                filtered_outages.append(df_state)
        except Exception as e:
            print(f"Failed to read {file_path}: {e}")

    if not filtered_outages:
        print(f"No outage records found for states: {states}")
        return pd.DataFrame()

    outages_df = pd.concat(filtered_outages, ignore_index=True)
    outages_df = outages_df.rename(columns={'run_start_time': 'outages_start_time'})

    print(f"Finished processing outages for {states}. Shape: {outages_df.shape}")
    return outages_df


# Multiple states
# tx_fl_outages_df = prepare_outages_dataset(["Texas", "Florida"], outages_dir)
top_least_10_states_outages_df = prepare_outages_dataset(top_least_10_outages_states, outages_dir)


Processing outages for ['Florida', 'California', 'Texas', 'Louisiana', 'Michigan', 'Montana', 'South Dakota', 'District of Columbia', 'North Dakota', 'Wyoming']: 100%|██████████| 10/10 [01:24<00:00,  8.42s/file]


Finished processing outages for ['Florida', 'California', 'Texas', 'Louisiana', 'Michigan', 'Montana', 'South Dakota', 'District of Columbia', 'North Dakota', 'Wyoming']. Shape: (50998862, 5)


In [13]:
top_least_10_states_outages_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50998862 entries, 0 to 50998861
Data columns (total 5 columns):
 #   Column              Dtype         
---  ------              -----         
 0   fips_code           int64         
 1   county              object        
 2   state               object        
 3   customers_out       float64       
 4   outages_start_time  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 1.9+ GB


- This is a fairer size, though we will still take it further down during modelling

In [14]:
top_least_10_states_outages_df

,fips_code,county,state,customers_out,outages_start_time
0,6029,Kern,California,30.0,2014-11-01 04:00:00
1,6037,Los Angeles,California,1555.0,2014-11-01 04:00:00
2,6065,Riverside,California,2.0,2014-11-01 04:00:00
3,6113,Yolo,California,1.0,2014-11-01 04:00:00
4,12011,Broward,Florida,17.0,2014-11-01 04:00:00
...,...,...,...,...,...
50998857,48473,Waller,Texas,2.0,2023-12-31 23:45:00
50998858,48481,Wharton,Texas,33.0,2023-12-31 23:45:00
50998859,48489,Willacy,Texas,4.0,2023-12-31 23:45:00
50998860,48491,Williamson,Texas,1.0,2023-12-31 23:45:00



---  
### 2.2 Load Storm Events Data

- Read .csv files in ../dynamic_rhythm_env/NOAA_StormEvents/
- Inspect structure, relevant columns like event_type, state, begin_date_time

In [15]:
import pandas as pd

# Load storm events file (2014–2024)
storm_events_path = '../dynamic_rhythm_train_data/NOAA_StormEvents/StormEvents_2014_2024.csv'
storm_df = pd.read_csv(storm_events_path)

# Group by state to get total number of storm events
storm_summary = storm_df.groupby('STATE').agg(
    total_events=('EVENT_ID', 'count'),
    total_deaths=('DEATHS_DIRECT', 'sum'),
    total_injuries=('INJURIES_DIRECT', 'sum')
).sort_values(by='total_events', ascending=False)

# Top 5 states by storm event frequency
top_5_storm_states = storm_summary.head(5)

# Bottom 5 states (least number of storm events)
least_5_storm_states = storm_summary.tail(5)

# Display
print("List of Top 5 States by Storm Event:", top_5_storm_states.index.tolist())
display(top_5_storm_states)

print("List of Least 5 States by Storm Event:", least_5_storm_states.index.tolist())
display(least_5_storm_states)


List of Top 5 States by Storm Event: ['TEXAS', 'KANSAS', 'VIRGINIA', 'IOWA', 'CALIFORNIA']


,total_events,total_deaths,total_injuries
STATE,,,
TEXAS,52039,625,3297
KANSAS,25616,22,159
VIRGINIA,22665,54,242
IOWA,22610,38,280
CALIFORNIA,22467,472,1164


List of Least 5 States by Storm Event: ['E PACIFIC', 'ST LAWRENCE R', 'HAWAII WATERS', 'GULF OF ALASKA', 'GUAM WATERS']


,total_events,total_deaths,total_injuries
STATE,,,
E PACIFIC,72,9,6
ST LAWRENCE R,17,0,0
HAWAII WATERS,14,1,1
GULF OF ALASKA,6,0,0
GUAM WATERS,1,0,0



---

**Top 5 States (by total storm events)**
These are strong candidates for outage prediction based on weather:

| State      | Events | Deaths | Injuries |
|------------|--------|--------|----------|
| **Texas**      | 52,039 | 625    | 3,297     |
| **Kansas**     | 25,616 | 22     | 159       |
| **Virginia**   | 22,665 | 54     | 242       |
| **Iowa**       | 22,610 | 38     | 280       |
| **California** | 22,467 | 472    | 1,164     |

These states are **diverse in region and risk type** — from hurricanes (TX, VA) to tornadoes and blizzards (KS, IA), and wildfires (CA).

---

**Least 5 Regions**
| Region            | Events | Deaths | Injuries |
|-------------------|--------|--------|----------|
| E PACIFIC         | 72     | 9      | 6        |
| ST LAWRENCE R     | 17     | 0      | 0        |
| HAWAII WATERS     | 14     | 1      | 1        |
| GULF OF ALASKA    | 6      | 0      | 0        |
| GUAM WATERS       | 1      | 0      | 0        |

These are **offshore or territorial zones** and may not align well with outage records — good candidates for exclusion or generalisation.

---

Now we've filtered **Top/Least 5 States** by:
- `total_outages` and `total_customers_out` (from outage data)
- `total_events` and `total_deaths/injuries` (from storm data)

We now have a **good shortlist of 10–15 priority states** for analysis, modeling, and visual exploration.


In [16]:
top_least_10_storms_states = least_5_storm_states.index.tolist() + top_5_storm_states.index.tolist()
print(top_least_10_storms_states)

['E PACIFIC', 'ST LAWRENCE R', 'HAWAII WATERS', 'GULF OF ALASKA', 'GUAM WATERS', 'TEXAS', 'KANSAS', 'VIRGINIA', 'IOWA', 'CALIFORNIA']

['E PACIFIC', 'ST LAWRENCE R', 'HAWAII WATERS', 'GULF OF ALASKA', 'GUAM WATERS', 'TEXAS', 'KANSAS', 'VIRGINIA', 'IOWA', 'CALIFORNIA']


['E PACIFIC',
 'ST LAWRENCE R',
 'HAWAII WATERS',
 'GULF OF ALASKA',
 'GUAM WATERS',
 'TEXAS',
 'KANSAS',
 'VIRGINIA',
 'IOWA',
 'CALIFORNIA']

In [17]:


# Read the storm events CSV
storm_events = pd.read_csv(storm_events_path)

# Filter for the selected states
top_least_10_storms_states_df = storm_events[storm_events['STATE'].isin(top_least_10_storms_states)]

# Show a summary of the filtered data
print(f"Finished filtering storm events. Shape: {top_least_10_storms_states_df.shape}")
print(top_least_10_storms_states_df.info())

top_least_10_storms_states_df


Finished filtering storm events. Shape: (145507, 51)
<class 'pandas.core.frame.DataFrame'>
Index: 145507 entries, 8 to 691428
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   BEGIN_YEARMONTH     145507 non-null  int64  
 1   BEGIN_DAY           145507 non-null  int64  
 2   BEGIN_TIME          145507 non-null  int64  
 3   END_YEARMONTH       145507 non-null  int64  
 4   END_DAY             145507 non-null  int64  
 5   END_TIME            145507 non-null  int64  
 6   EPISODE_ID          145507 non-null  int64  
 7   EVENT_ID            145507 non-null  int64  
 8   STATE               145507 non-null  object 
 9   STATE_FIPS          145507 non-null  int64  
 10  YEAR                145507 non-null  int64  
 11  MONTH_NAME          145507 non-null  object 
 12  EVENT_TYPE          145507 non-null  object 
 13  CZ_TYPE             145507 non-null  object 
 14  CZ_FIPS             145507 non-null 

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
8,201403,16,1400,201403,17,1200,83806,507869,VIRGINIA,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure ejecting out of the Southern Plai...,Various sources reported snowfall totals rangi...,CSV
9,201403,13,0,201403,13,1100,84127,507867,VIRGINIA,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Northwest winds behind a departing cold front,...",Several minor accidents occurred in Tazewell c...,CSV
10,201405,11,2230,201405,11,2230,83868,506362,TEXAS,48,...,2.0,N,LAKEVIEW,34.6990,-100.7000,34.6990,-100.7000,A line of thunderstorm activity blossomed acro...,Numerous large tree limbs were blown down nort...,CSV
16,201403,25,400,201403,25,1800,83807,505969,VIRGINIA,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Despite a surface coastal storm that was well ...,Snowfall reports ranged from 3 to 4 inches acr...,CSV
17,201403,25,400,201403,25,1800,83807,505970,VIRGINIA,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Despite a surface coastal storm that was well ...,Snowfall reports ranged from less than 1 up to...,CSV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691413,202406,4,1600,202406,4,1601,192700,1189543,KANSAS,20,...,4.0,NNE,INMAN,38.2900,-97.7400,38.2900,-97.7400,A weak cold front pushed south across the area...,Broadcast media (KAKE TV) meteorologist report...,CSV
691418,202406,7,1745,202406,7,1805,192915,1190968,KANSAS,20,...,2.0,WNW,HOXIE,39.3645,-100.4708,39.3645,-100.4708,During the afternoon to early evening hours a ...,Hail ranging in size from dime to golf ball fe...,CSV
691419,202406,7,1635,202406,7,1635,192915,1190967,KANSAS,20,...,0.0,NW,KANORADO,39.3330,-102.0335,39.3330,-102.0335,During the afternoon to early evening hours a ...,Pea to quarter sized hail reported.,CSV
691420,202406,7,1730,202406,7,1730,192915,1190966,KANSAS,20,...,2.0,NW,SEGUIN,39.3576,-100.6091,39.3576,-100.6091,During the afternoon to early evening hours a ...,Ping pong ball to golf ball sized hail fell ov...,CSV


In [18]:
# target_state = 'TEXAS'

# # Path to the storm events CSV
# storm_events_path = '../dynamic_rhythm_train_data/NOAA_StormEvents/StormEvents_2014_2024.csv'

# # Read storm events
# storm_events = pd.read_csv(storm_events_path)

# # Filter for the same state
# storm_events_14_24_df = storm_events[storm_events['STATE'] == target_state.upper()]

# print(f"Finished reading storm events. Shape: {storm_events_14_24_df.shape}")

# storm_events_14_24_df.info()


In [19]:
top_least_10_storms_states_df.head(2)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
8,201403,16,1400,201403,17,1200,83806,507869,VIRGINIA,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure ejecting out of the Southern Plai...,Various sources reported snowfall totals rangi...,CSV
9,201403,13,0,201403,13,1100,84127,507867,VIRGINIA,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Northwest winds behind a departing cold front,...",Several minor accidents occurred in Tazewell c...,CSV


In [20]:
top_least_10_states_outages_df.head(2)


,fips_code,county,state,customers_out,outages_start_time
0,6029,Kern,California,30.0,2014-11-01 04:00:00
1,6037,Los Angeles,California,1555.0,2014-11-01 04:00:00


- keeping state selection consistent across both datasets by basing it entirely on the power outages data ensures clean joins, aligned time series, and avoids inconsistencies caused by extra regions or naming issues in the storms data.

- let's use the outages states to filter the storms

In [21]:
# print(storm_events['STATE'].unique())
# print(outages_selected_df['state'].nunique())
print(top_least_10_outages_states)

['Florida', 'California', 'Texas', 'Louisiana', 'Michigan', 'Montana', 'South Dakota', 'District of Columbia', 'North Dakota', 'Wyoming']


- 'District Of Columbia' instead of 'District of Columbia' in storms

In [ ]:
# storm_events['STATE'].unique()

In [ ]:
storm_events['STATE'] = storm_events['STATE'].str.title()

storm_events['STATE'] = storm_events['STATE'].replace({
    'District Of Columbia':'District of Columbia'
})

# storm_events['STATE'].unique()

In [24]:


# Now filter storm data to match only outages-based states
storm_from_outages_states_df = storm_events[storm_events['STATE'].isin(top_least_10_outages_states)]

# print(top_least_10_outages_states)
# storm_from_outages_states_df['STATE'].unique()

intersectings_states = set(storm_from_outages_states_df['STATE'].unique()) & set(top_least_10_outages_states)
print(intersectings_states)

# storm_from_outages_states_df

{'South Dakota', 'Texas', 'Michigan', 'California', 'Louisiana', 'Montana', 'North Dakota', 'Wyoming', 'District of Columbia', 'Florida'}


In [25]:
storm_from_outages_states_df['STATE'].unique()

array(['Texas', 'California', 'Wyoming', 'Louisiana', 'Montana',
       'South Dakota', 'North Dakota', 'Florida', 'Michigan',
       'District of Columbia'], dtype=object)

In [ ]:
# storm_from_outages_states_df['BEGIN_DATE_TIME'].unique()

# top_least_10_outages_states

Why only 7 states were initially fetched instead of the full 10:

---

Our `top_least_10_outages_states` includes:
- `'District of Columbia'`
- `'US Virgin Islands'`
- `'United States Virgin Islands'`

---

In `storm_events['STATE'].unique()`:
- `'District Of Columbia'` Match (but with title case difference)
- `'Virgin Islands'` Exists, but your dataset has inconsistent naming (`US Virgin Islands`, `United States Virgin Islands`)

---

**Why 3 states didn’t match:**
1. `'District of Columbia'` ≠ `'District Of Columbia'` → fix with consistent title-casing.
2. `'United States Virgin Islands'` ≠ `'Virgin Islands'`
3. `'US Virgin Islands'` ≠ `'Virgin Islands'`

---


In [ ]:
# storm_from_outages_states_df['DEATHS_INDIRECT'].unique()
# top_least_10_states_outages_df.info()

In [56]:
# storm_from_outages_states_df['EVENT_TYPE'].unique()


In [68]:
def parse_damage(damage_str):
    if pd.isnull(damage_str):
        return 0
    if isinstance(damage_str, (int, float)):
        return damage_str
    damage_str = damage_str.strip()
    multiplier = 1
    if damage_str.endswith('K'):
        multiplier = 1_000
        damage_str = damage_str[:-1]
    elif damage_str.endswith('M'):
        multiplier = 1_000_000
        damage_str = damage_str[:-1]
    elif damage_str.endswith('B'):
        multiplier = 1_000_000_000
        damage_str = damage_str[:-1]
    try:
        return float(damage_str) * multiplier
    except ValueError:
        return 0


def prepare_daily_storm_outage_features(storm_df, outage_df):
    # Ensure datetime fields are parsed
    storm_df['BEGIN_DATETIME'] = pd.to_datetime(storm_df['BEGIN_DATETIME'])
    outage_df['outages_start_time'] = pd.to_datetime(outage_df['outages_start_time'])

    # Add event_day
    storm_df['event_day'] = storm_df['BEGIN_DATETIME'].dt.date
    outage_df['event_day'] = outage_df['outages_start_time'].dt.date

    storm_df['DAMAGE_PROPERTY'] = storm_df['DAMAGE_PROPERTY'].apply(parse_damage)
    storm_df['DAMAGE_CROPS'] = storm_df['DAMAGE_CROPS'].apply(parse_damage)

    # Count of each event type per state and day
    event_type_counts = (
        storm_df.groupby(['STATE', 'event_day', 'EVENT_TYPE'])
        .size()
        .reset_index(name='event_count')
        .pivot(index=['STATE', 'event_day'], columns='EVENT_TYPE', values='event_count')
        .fillna(0)
        .astype(int)
    )

    # Rename columns
    event_type_counts.columns = [
        f"num_{col.lower().replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '').replace('-', '').replace('__', '_')}_events"
        for col in event_type_counts.columns
    ]
    event_type_counts = event_type_counts.reset_index()

    # Basic storm aggregation
    storm_daily_df = (
        storm_df
        .groupby(['STATE', 'event_day'])
        .agg(
            storm_event_count=('EVENT_ID', 'count'),
            deaths_total=('DEATHS_DIRECT', 'sum'),
            deaths_indirect=('DEATHS_INDIRECT', 'sum'),
            injuries_indirect=('INJURIES_INDIRECT', 'sum'),
            injuries_direct=('DEATHS_DIRECT', 'sum'),
            total_damage_property=('DAMAGE_PROPERTY','sum'),
            total_damage_crops=('DAMAGE_CROPS','sum')
            # total_damage_property=('DAMAGE_PROPERTY', lambda x: x.dropna().count())
        )
        .reset_index()
    )

    # Merge event type features
    storm_daily_df = storm_daily_df.merge(event_type_counts, on=['STATE', 'event_day'], how='left')

    # Daily outage aggregation
    outage_daily_df = (
        outage_df
        .groupby(['state', 'event_day'])
        .agg(
            total_outages=('fips_code', 'count'),
            total_customers_out=('customers_out', 'sum'),
            avg_customers_out=('customers_out', 'mean')
        )
        .reset_index()
    )
    outage_daily_df['state'] = outage_daily_df['state'].str.title()

    # Merge on state and day
    merged_df = pd.merge(
        storm_daily_df,
        outage_daily_df,
        left_on=['STATE', 'event_day'],
        right_on=['state', 'event_day'],
        how='inner'
    )

    # Convert event_day to datetime
    merged_df['event_day'] = pd.to_datetime(merged_df['event_day'])

    # Add time features
    merged_df['year'] = merged_df['event_day'].dt.year
    merged_df['month'] = merged_df['event_day'].dt.month
    merged_df['day'] = merged_df['event_day'].dt.day
    merged_df['dayofweek'] = merged_df['event_day'].dt.dayofweek
    merged_df['week'] = merged_df['event_day'].dt.isocalendar().week
    merged_df['quarter'] = merged_df['event_day'].dt.quarter

    # Cyclical encodings
    merged_df['dayofweek_sin'] = np.sin(2 * np.pi * merged_df['dayofweek'] / 7)
    merged_df['dayofweek_cos'] = np.cos(2 * np.pi * merged_df['dayofweek'] / 7)
    merged_df['month_sin'] = np.sin(2 * np.pi * merged_df['month'] / 12)
    merged_df['month_cos'] = np.cos(2 * np.pi * merged_df['month'] / 12)

    # Sort for lag features
    merged_df = merged_df.sort_values(['STATE', 'event_day'])

    # Lag and rolling features
    merged_df['lag_1_storms'] = merged_df.groupby('STATE')['storm_event_count'].shift(1)
    merged_df['rolling_3d_storms'] = (
        merged_df.groupby('STATE')['storm_event_count']
        .rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)
    )

    merged_df['lag_1_outages'] = merged_df.groupby('STATE')['total_outages'].shift(1)
    merged_df['rolling_3d_outages'] = (
        merged_df.groupby('STATE')['total_outages']
        .rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)
    )

    # Binary target
    merged_df['outage_occurred'] = (merged_df['total_outages'] > 0).astype(int)

    # Outage severity categorization
    def categorize_severity(x):
        if x == 0:
            return 'None'
        elif x < 1000:
            return 'Low'
        elif x < 10000:
            return 'Medium'
        else:
            return 'High'

    merged_df['outage_severity'] = merged_df['total_customers_out'].apply(categorize_severity)

    return merged_df


# Call the function on your datasets
merged_daily_df = prepare_daily_storm_outage_features(
    storm_df=storm_from_outages_states_df,
    outage_df=top_least_10_states_outages_df
)

merged_daily_df.head()


/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_1724/1989978007.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storm_df['BEGIN_DATETIME'] = pd.to_datetime(storm_df['BEGIN_DATETIME'])
/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_1724/1989978007.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storm_df['event_day'] = storm_df['BEGIN_DATETIME'].dt.date
/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_1724/1989978007.py:32: SettingWithCopyWarning: 
A value is trying 

,STATE,event_day,storm_event_count,deaths_total,deaths_indirect,injuries_indirect,injuries_direct,total_damage_property,total_damage_crops,num_astronomical_low_tide_events,...,dayofweek_sin,dayofweek_cos,month_sin,month_cos,lag_1_storms,rolling_3d_storms,lag_1_outages,rolling_3d_outages,outage_occurred,outage_severity
0,California,2014-11-01,18,0,0,0,0,0.0,0.0,0,...,-0.974928,-0.222521,-0.5,0.866025,NaN,18.000000,NaN,738.000000,1,High
1,California,2014-11-03,2,0,0,0,0,0.0,0.0,0,...,0.000000,1.000000,-0.5,0.866025,18.0,10.000000,738.0,762.500000,1,High
2,California,2014-11-04,2,0,0,0,0,0.0,0.0,0,...,0.781831,0.623490,-0.5,0.866025,2.0,7.333333,787.0,744.666667,1,High
3,California,2014-11-07,5,0,0,0,0,0.0,0.0,0,...,-0.433884,-0.900969,-0.5,0.866025,2.0,3.000000,709.0,761.666667,1,High
4,California,2014-11-08,1,0,0,0,0,0.0,0.0,0,...,-0.974928,-0.222521,-0.5,0.866025,5.0,2.666667,789.0,715.666667,1,High


In [69]:
# merged_daily_df['total_damage_property'].unique()
merged_daily_df['state'].unique()

array(['California', 'Florida', 'Louisiana', 'Michigan', 'Montana',
       'North Dakota', 'South Dakota', 'Texas', 'Wyoming'], dtype=object)

In [58]:
# merged_daily_df.columns

In [26]:

def make_ts_states_outages(states,
                    start_year,
                    start_month,
                    start_day,
                    end_year,
                    end_month,
                    end_day,
                    df_power):
    """
    Create a time series DataFrame for power outages for multiple states within a date range.

    Parameters:
    - states (list of str): List of state names to filter (case-sensitive).
    - start_year, start_month, start_day, end_year, end_month, end_day (int): Date range.
    - df_power (DataFrame): Pre-loaded and pre-filtered outage data.

    Returns:
    - DataFrame indexed by time with 'customers_out' and 'state' fields.
    """

    # Ensure datetime format
    df_power['outages_start_time'] = pd.to_datetime(df_power['outages_start_time'])

    # Filter by date range
    start_date = pd.Timestamp(year=start_year, month=start_month, day=start_day)
    end_date = pd.Timestamp(year=end_year, month=end_month, day=end_day)

    df_power_filtered = df_power[
        (df_power['outages_start_time'] >= start_date) & 
        (df_power['outages_start_time'] <= end_date) &
        (df_power['state'].isin(states))
    ].copy()

    # Create time series DataFrame
    df_power_filtered['time'] = df_power_filtered['outages_start_time']
    df_power_ts = df_power_filtered[['time', 'state', 'customers_out']]

    return df_power_ts


def make_ts_states_storms(states, event_types, start_year, start_month, start_day, end_year, end_month, end_day, df):
    """
    Construct a DataFrame with 15-minute intervals indicating event occurrence for multiple states.

    Parameters:
    - states (list of str): List of states to filter (e.g., ["Texas", "California"]).
    - event_types (list): The event types to filter (e.g., ["Winter Storm", "Hurricane"]).
    - start_year, start_month, start_day, end_year, end_month, end_day (int): Date range.
    - df (pd.DataFrame): The NOAA StormEvent database.

    Returns:
    - pd.DataFrame: A DataFrame with 15-minute intervals, event counts, and state label.
    """

    from datetime import datetime

    # Generate the time range for the new DataFrame
    start_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day, 23, 45)
    time_index = pd.date_range(start=start_date, end=end_date, freq='15min')

    # Convert BEGIN and END times into datetime objects
    df['BEGIN_DATETIME'] = pd.to_datetime(
        df['BEGIN_YEARMONTH'].astype(str) + df['BEGIN_DAY'].astype(str).str.zfill(2) +
        df['BEGIN_TIME'].astype(str).str.zfill(4), format='%Y%m%d%H%M'
    )
    df['END_DATETIME'] = pd.to_datetime(
        df['END_YEARMONTH'].astype(str) + df['END_DAY'].astype(str).str.zfill(2) +
        df['END_TIME'].astype(str).str.zfill(4), format='%Y%m%d%H%M'
    )

    all_states_df = []

    for state in states:
        # Create state-specific frame
        new_df = pd.DataFrame({'time': time_index})
        for event_type in event_types:
            new_df[f'event_count {event_type}'] = 0

        # Filter for this state and event types
        filtered_df = df[
            (df['STATE'] == state) &
            (df['EVENT_TYPE'].isin(event_types)) &
            (df['END_DATETIME'] >= start_date) &
            (df['BEGIN_DATETIME'] <= end_date)
        ].copy()

        # Iterate through events to populate counts
        for event_type in event_types:
            event_subset = filtered_df[filtered_df['EVENT_TYPE'] == event_type]

            for _, row in event_subset.iterrows():
                event_start = row['BEGIN_DATETIME'].round('15min')
                event_end = row['END_DATETIME'].round('15min')
                start_idx = new_df['time'].searchsorted(event_start)
                end_idx = new_df['time'].searchsorted(event_end)

                if start_idx < len(new_df) and end_idx <= len(new_df):
                    new_df.loc[start_idx:end_idx, f'event_count {event_type}'] += 1

        new_df['STATE'] = state
        all_states_df.append(new_df)

    # Combine all states
    combined_df = pd.concat(all_states_df, ignore_index=True)

    return combined_df


In [27]:
top_least_10_states_outages_df['outages_start_time'].min(), top_least_10_states_outages_df['outages_start_time'].max()

(Timestamp('2014-11-01 04:00:00'), Timestamp('2023-12-31 23:45:00'))

In [28]:
top_least_10_storm_types = top_least_10_storms_states_df['EVENT_TYPE'].unique()
print("number of storms types in the top 5 and least 5 states:", top_least_10_storms_states_df['EVENT_TYPE'].nunique(), "\n")
print(top_least_10_storm_types)

number of storms types in the top 5 and least 5 states: 49 

['Winter Storm' 'Winter Weather' 'Thunderstorm Wind' 'Heavy Snow'
 'High Wind' 'Hail' 'Drought' 'Tornado' 'Dust Storm' 'Flash Flood' 'Flood'
 'Blizzard' 'Cold/Wind Chill' 'Freezing Fog' 'Rip Current' 'Strong Wind'
 'Dense Fog' 'Sleet' 'Debris Flow' 'High Surf' 'Frost/Freeze' 'Lightning'
 'Coastal Flood' 'Dust Devil' 'Wildfire' 'Heavy Rain' 'Funnel Cloud'
 'Waterspout' 'Heat' 'Extreme Cold/Wind Chill' 'Ice Storm'
 'Tropical Storm' 'Astronomical Low Tide' 'Marine Strong Wind'
 'Sneakerwave' 'Seiche' 'Excessive Heat' 'Marine High Wind' 'Tsunami'
 'Tropical Depression' 'Storm Surge/Tide' 'Avalanche' 'Marine Dense Fog'
 'Marine Thunderstorm Wind' 'Hurricane' 'Marine Hail' 'Dense Smoke'
 'Lake-Effect Snow' 'Marine Tropical Depression']


In [29]:
outages_ts_df = make_ts_states_outages(
    states=top_least_10_outages_states,
    start_year=2014, start_month=1, start_day=1,
    end_year=2023, end_month=12, end_day=31,
    df_power=top_least_10_states_outages_df
)

# ts_outages_df

In [46]:
# outages_ts_df['state'].unique()
outages_ts_df.info()

# top_least_10_storms_states
# len(top_least_10_storm_types)
# top_least_10_storms_states_df['EVENT_TYPE'].nunique()

# print(storm_from_outages_states_df['EVENT_TYPE'].unique())
# print(top_least_10_storm_types)

<class 'pandas.core.frame.DataFrame'>
Index: 50986284 entries, 0 to 50986283
Data columns (total 3 columns):
 #   Column         Dtype         
---  ------         -----         
 0   time           datetime64[ns]
 1   state          object        
 2   customers_out  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.5+ GB


In [32]:
storm_ts_df = make_ts_states_storms(
    states=storm_from_outages_states_df['STATE'].unique(),
    event_types=storm_from_outages_states_df['EVENT_TYPE'].unique(),
    start_year=2014, start_month=1, start_day=1,
    end_year=2023, end_month=12, end_day=31,
    df=storm_from_outages_states_df
)

# storm_ts_df

/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_1724/2948589550.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BEGIN_DATETIME'] = pd.to_datetime(
/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_1724/2948589550.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['END_DATETIME'] = pd.to_datetime(


In [34]:
storm_ts_df.info()
# storm_ts_df['STATE'].unique()
# top_least_10_storms_states_df['STATE'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3505920 entries, 0 to 3505919
Data columns (total 46 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   time                                 datetime64[ns]
 1   event_count Thunderstorm Wind        int64         
 2   event_count Hail                     int64         
 3   event_count Drought                  int64         
 4   event_count Winter Weather           int64         
 5   event_count Heavy Snow               int64         
 6   event_count Winter Storm             int64         
 7   event_count Ice Storm                int64         
 8   event_count Flood                    int64         
 9   event_count Avalanche                int64         
 10  event_count Tornado                  int64         
 11  event_count Blizzard                 int64         
 12  event_count High Wind                int64         
 13  event_count Extreme Cold/Wi

In [ ]:
# df_state_ts_power.columns
# df_state_ts_power

In [ ]:
# df_model['num_drought_events'].describe()

In [ ]:
outages_ts_df

,time,state,customers_out
0,2014-11-01 04:00:00,California,30.0
1,2014-11-01 04:00:00,California,1555.0
2,2014-11-01 04:00:00,California,2.0
3,2014-11-01 04:00:00,California,1.0
4,2014-11-01 04:00:00,Florida,17.0
...,...,...,...
50986279,2023-12-31 00:00:00,Texas,19.0
50986280,2023-12-31 00:00:00,Texas,4.0
50986281,2023-12-31 00:00:00,Texas,161.0
50986282,2023-12-31 00:00:00,Wyoming,1.0


In [35]:
outages_ts_df.describe().T

,count,mean,min,25%,50%,75%,max,std
time,50986284,2020-03-16 16:13:33.389327616,2014-11-01 04:00:00,2018-07-21 23:30:00,2020-06-15 14:00:00,2022-03-27 08:15:00,2023-12-31 00:00:00,NaN
customers_out,48855271.0,294.938671,0.0,2.0,6.0,44.0,1777800.0,4724.017568


In [ ]:
# df_state_ts_events.columns
# storm_ts_df.describe().T

In [ ]:
# df_combined_hr, df_combined_day = combine_agg_ts(...)

# storm_events_14_24_df.columns


In [ ]:
print(storm_ts_df['STATE'].unique(),"\n")
print(outages_ts_df['state'].unique())


['Texas' 'California' 'Wyoming' 'Louisiana' 'Montana' 'South Dakota'
 'North Dakota' 'Florida' 'Michigan' 'District of Columbia'] 

['California' 'Florida' 'Louisiana' 'Michigan' 'Texas' 'North Dakota'
 'District of Columbia' 'Montana' 'South Dakota' 'Wyoming']


In [ ]:
# Step 1: Copy the data
df_labels = top_least_10_states_outages_df.copy()

# Step 2: Parse run_start_time into date
df_labels['date'] = pd.to_datetime(df_labels['outages_start_time']).dt.date

# Step 3: Group by fips_code, county, and date
df_outages_grouped = (
    df_labels
    .groupby(['fips_code', 'county', 'date','state','outages_start_time'])
    .agg({'customers_out': 'sum'})
    .reset_index()
)

# Step 4: Create the binary major outage label
# THRESHOLD = 1000  # customers
# df_outages_grouped['major_outage'] = (df_outages_grouped['customers_out'] >= THRESHOLD).astype(int)

# df_outages_grouped['fips_code'] = df_outages_grouped['fips_code'].astype(str)
# df_outages_grouped['date'] = pd.to_datetime(df_outages_grouped['date'])

# Step 5: View a few rows
df_outages_grouped.head()


,fips_code,county,date,state,outages_start_time,customers_out,major_outage
0,6001,Alameda,2017-11-21,California,2017-11-21 00:00:00,28.0,0
1,6001,Alameda,2018-06-01,California,2018-06-01 05:00:00,0.0,0
2,6001,Alameda,2018-06-05,California,2018-06-05 23:30:00,107.0,0
3,6001,Alameda,2018-06-05,California,2018-06-05 23:45:00,108.0,0
4,6001,Alameda,2018-06-06,California,2018-06-06 00:00:00,66.0,0


In [ ]:
        #    Total_Injuries=('INJURIES_DIRECT', 'sum'),
        #     Total_Deaths=('DEATHS_DIRECT', 'sum'),
        #     Total_Property_Damage=('DAMAGE_PROPERTY', 'sum'),
df_outages_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50998862 entries, 0 to 50998861
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   fips_code           object        
 1   county              object        
 2   date                datetime64[ns]
 3   state               object        
 4   outages_start_time  datetime64[ns]
 5   customers_out       float64       
 6   major_outage        int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 2.7+ GB


In [ ]:
# top_least_10_storms_states_df['CZ_NAME'].unique()

In [ ]:
# top_least_10_storms_states_df.head().T

In [ ]:

df_storms = storm_from_outages_states_df.copy()

# Fix the date
df_storms['date'] = pd.to_datetime(df_storms['BEGIN_DATE_TIME']).dt.date

# Create fips_code
df_storms['STATE_FIPS'] = df_storms['STATE_FIPS'].astype(str).str.zfill(2)
df_storms['CZ_FIPS'] = df_storms['CZ_FIPS'].astype(str).str.zfill(3)
df_storms['fips_code'] = df_storms['STATE_FIPS'] + df_storms['CZ_FIPS']

# --- Clean the damage columns here ---
df_storms['DAMAGE_PROPERTY'] = df_storms['DAMAGE_PROPERTY'].apply(parse_damage)
df_storms['DAMAGE_CROPS'] = df_storms['DAMAGE_CROPS'].apply(parse_damage)

df_storms = df_storms.rename(columns={
    'STATE': 'state',
})

storm_agg_dict = {
    'EVENT_TYPE': 'count',
    'DAMAGE_PROPERTY': 'sum',
    'DAMAGE_CROPS': 'sum',
    'INJURIES_DIRECT': 'sum',
    'INJURIES_INDIRECT': 'sum',
    'DEATHS_DIRECT': 'sum',
    'TOR_F_SCALE': pd.Series.nunique,
}

storm_agg_renames = {
    'EVENT_TYPE': 'num_events',
    'DAMAGE_PROPERTY': 'total_property_damage',
    'DAMAGE_CROPS': 'total_crop_damage',
    'TOR_F_SCALE': 'num_tornado_scales',
    'INJURIES_DIRECT': 'num_direct_injuries',
    'INJURIES_INDIRECT': 'num_indirect_injuries',
    'DEATHS_DIRECT': 'num_direct_deaths',
}


# --- THEN group ---
df_storms_grouped = (
    df_storms
    .groupby(['fips_code', 'date', 'state'])
    .agg(storm_agg_dict)
    .rename(columns=storm_agg_renames)
    .reset_index()
)

 
#  fips_code should be string in both
df_storms_grouped['fips_code'] = df_storms_grouped['fips_code'].astype(str)

# date should be datetime in both
df_storms_grouped['date'] = pd.to_datetime(df_storms_grouped['date'])


/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_69472/2266566031.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_storms['date'] = pd.to_datetime(df_storms['BEGIN_DATE_TIME']).dt.date


In [ ]:
storm_from_outages_states_df[['DAMAGE_PROPERTY','DAMAGE_CROPS','INJURIES_DIRECT','DEATHS_DIRECT']].sample(5)

,DAMAGE_PROPERTY,DAMAGE_CROPS,INJURIES_DIRECT,DEATHS_DIRECT
501864,NaN,NaN,0,0
562407,0.00K,0.00K,0,0
197296,0.00K,0.00K,0,0
447346,0.00K,0.00K,0,0
475457,0.00K,0.00K,0,0


In [ ]:
# df_storms_grouped.sample(5)
df_storms_grouped.describe()

,date,num_events,total_property_damage,total_crop_damage,num_direct_injuries,num_indirect_injuries,num_direct_deaths,num_tornado_scales
count,109643,109643.000000,1.096430e+05,1.096430e+05,109643.000000,109643.000000,109643.00000,109643.000000
mean,2019-11-17 20:19:11.942212352,1.509490,1.591601e+06,7.415820e+04,0.052571,0.010580,0.01628,0.026194
min,2014-01-01 00:00:00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,0.000000
25%,2017-03-07 00:00:00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,0.000000
50%,2020-04-20 00:00:00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,0.000000
75%,2022-08-05 00:00:00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,0.000000
max,2024-09-30 00:00:00,82.000000,1.700000e+10,1.500000e+09,806.000000,55.000000,86.00000,4.000000
std,NaN,1.596529,8.691500e+07,5.138292e+06,3.349964,0.410721,0.42912,0.180060


In [ ]:
# Check for non-numeric values (should be none)
print("non-numerics:",df_storms_grouped['total_property_damage'].apply(lambda x: isinstance(x, str)).sum())
print("non-numerics:",df_storms_grouped['total_crop_damage'].apply(lambda x: isinstance(x, str)).sum())

# Property damage greater than 1 billion
gt_onebillion_property = df_storms_grouped[df_storms_grouped['total_property_damage'] > 1e9]['total_property_damage'].unique()

# Crop damage greater than 1 billion
gt_onebillion_crop = df_storms_grouped[df_storms_grouped['total_crop_damage'] > 1e9]['total_crop_damage'].unique()

print("Extreme property damage amount entries (over 1 billion):")
print(gt_onebillion_property)

print("\nExtreme crop damage amount amount entries (over 1 billion):")
print(gt_onebillion_crop)


non-numerics: 0
non-numerics: 0
Extreme property damage amount entries (over 1 billion):
[1.500000e+09 1.700000e+10 2.000000e+09 2.204500e+09 6.700000e+09
 3.000000e+09 7.000000e+09 1.070000e+09 1.680000e+09 6.000000e+09
 1.250000e+09 1.340000e+09 1.000750e+09 1.100000e+09 1.360000e+09
 1.300200e+09 1.960025e+09 1.351270e+09 8.002830e+09 1.000000e+10
 1.000068e+10 1.300000e+09 1.950000e+09]

Extreme crop damage amount amount entries (over 1 billion):
[1.5e+09]


In [ ]:
df_storms_grouped = df_storms_grouped.rename(columns={
    'total_property_damage': 'total_property_damage_usd',
    'total_crop_damage': 'total_crop_damage_usd'
})
	

df_storms_grouped[['total_property_damage_usd','total_crop_damage_usd']].sample(2)

,total_property_damage_usd,total_crop_damage_usd
100423,0.0,0.0
48320,0.0,0.0


In [ ]:
# df_storms_grouped['total_crop_damage_usd'].unique()
# df_storms_grouped[['total_property_damage',
#        'total_crop_damage', 'num_tornado_scales']].sample(10)
# df_storms_grouped.columns

In [ ]:
df_storms_grouped['state'].unique()
df_outages_grouped['state'].unique()
# .info()


array(['California', 'District of Columbia', 'Florida', 'Louisiana',
       'Michigan', 'Montana', 'North Dakota', 'South Dakota', 'Texas',
       'Wyoming'], dtype=object)

In [ ]:
# df_outages_grouped.info()
# df_storms_grouped.info()


In [ ]:
# df_model = pd.merge(
#     df_outages_grouped,
#     df_storms_grouped,
#     on=['fips_code', 'date', 'state'],
#     how='inner'
# )

# df_model

In [ ]:
df_storms_grouped['fips_code'] = df_storms_grouped['fips_code'].astype(str)
df_storms_grouped['fips_code'] = df_storms_grouped['fips_code'].astype(str)
df_storms_grouped['date'] = pd.to_datetime(df_storms_grouped['date'])
df_storms_grouped['date'] = pd.to_datetime(df_storms_grouped['date'])


# --- Step 5A: Add per-event-type counts ---

# 1. Group storms by fips_code, date, and event_type
df_storms_event_counts = (
    df_storms
    .groupby(['fips_code', 'date', 'EVENT_TYPE','state'])
    .size()
    .reset_index(name='event_count')
)

# 2. Pivot so each event type becomes a separate column
df_storms_event_pivot = (
    df_storms_event_counts
    .pivot_table(index=['fips_code', 'date','state'], 
                 columns='EVENT_TYPE', 
                 values='event_count', 
                 fill_value=0)
    .reset_index()
)

df_storms_event_pivot['fips_code'] = df_storms_event_pivot['fips_code'].astype(str)
df_storms_event_pivot['fips_code'] = df_storms_event_pivot['fips_code'].astype(str)
df_storms_event_pivot['date'] = pd.to_datetime(df_storms_event_pivot['date'])
df_storms_event_pivot['date'] = pd.to_datetime(df_storms_event_pivot['date'])

# 3. Rename event columns to num_<event>_events for consistency
df_storms_event_pivot.columns = ['fips_code', 'date','state'] + [
     f"num_{col.lower().replace(' ', '_').replace('/', '_')}_events" for col in df_storms_event_pivot.columns[3:]
]


# 4. Merge with df_storms_grouped (the previous storm summary)
df_storms_full = pd.merge(
    df_storms_grouped,
    df_storms_event_pivot,
    on=['fips_code', 'date','state'],
    how='left'
)

# Fill NA event counts with 0 (if no such event that day)
event_cols = [col for col in df_storms_full.columns if col.startswith('num_') and col.endswith('_events')]
# df_storms_full[event_cols] = df_storms_full[event_cols].fillna(0)

# --- Final merge to outage labels ---

df_outages_grouped['fips_code'] = df_outages_grouped['fips_code'].astype(str)
df_outages_grouped['fips_code'] = df_outages_grouped['fips_code'].astype(str)
df_outages_grouped['date'] = pd.to_datetime(df_outages_grouped['date'])
df_outages_grouped['date'] = pd.to_datetime(df_outages_grouped['date'])

df_storms_full['fips_code'] = df_storms_full['fips_code'].astype(str)
df_storms_full['fips_code'] = df_storms_full['fips_code'].astype(str)
df_storms_full['date'] = pd.to_datetime(df_storms_full['date'])
df_storms_full['date'] = pd.to_datetime(df_storms_full['date'])

# Standardize fips_code
df_outages_grouped['fips_code'] = df_outages_grouped['fips_code'].astype(str).str.zfill(5).str.strip()
df_storms_full['fips_code'] = df_storms_full['fips_code'].astype(str).str.zfill(5).str.strip()

# Standardize date
df_outages_grouped['date'] = pd.to_datetime(df_outages_grouped['date']).dt.tz_localize(None)
df_storms_full['date'] = pd.to_datetime(df_storms_full['date']).dt.tz_localize(None)

# Standardize state
df_outages_grouped['state'] = df_outages_grouped['state'].str.strip().str.upper()
df_storms_full['state'] = df_storms_full['state'].str.strip().str.upper()

# Get the intersection of shared states
shared_states = set(df_outages_grouped['state'].unique()) & set(df_storms_full['state'].unique())

# Filter both DataFrames
df_outages_grouped = df_outages_grouped[df_outages_grouped['state'].isin(shared_states)]
df_storms_full = df_storms_full[df_storms_full['state'].isin(shared_states)]

df_model_inner = pd.merge(df_outages_grouped, df_storms_full, on=['fips_code', 'date', 'state'], how='inner')
df_model_fallback = pd.merge(df_outages_grouped, df_storms_full, on=['fips_code', 'date'], how='inner')

print(f"Inner merge rows: {len(df_model_inner)}")
print(f"Fallback merge rows (no state): {len(df_model_fallback)}")


# Merge storm data with outage labels
df_model = pd.merge(
    df_outages_grouped,
    df_storms_full,
    on=['fips_code', 'date','state'],
    how='inner'
)

# Optional safety fill again
df_model.fillna(0, inplace=True)

df_model.head()


: 

In [ ]:
# df_model['state_x'].unique()
df_model['date'].describe()

count                            30338
mean     2019-08-14 21:17:20.200408832
min                2014-11-01 00:00:00
25%                2018-03-06 00:00:00
50%                2019-12-07 12:00:00
75%                2021-06-21 00:00:00
max                2022-12-31 00:00:00
Name: date, dtype: object

In [ ]:
# Confirm overlap in fips_code
print("Overlap in fips_code:", len(set(df_outages_grouped['fips_code']) & set(df_storms_full['fips_code'])))

# Confirm overlap in state
print("Overlap in state:", len(set(df_outages_grouped['state']) & set(df_storms_full['state'])))

# Confirm overlap in date
print("Overlap in date:", len(set(df_outages_grouped['date']) & set(df_storms_full['date'])))

# Confirm exact match triplets
merge_keys_outages = set(zip(df_outages_grouped['fips_code'], df_outages_grouped['date'], df_outages_grouped['state']))
merge_keys_storms = set(zip(df_storms_full['fips_code'], df_storms_full['date'], df_storms_full['state']))
print("Exact matches (fips_code, date, state):", len(merge_keys_outages & merge_keys_storms))


Overlap in fips_code: 724
Overlap in state: 10
Overlap in date: 2915
Exact matches (fips_code, date, state): 30338


In [ ]:
print("df_outages_grouped states:",df_outages_grouped['state'].unique())
print("df_storms_full states:",df_storms_full['state'].unique())


df_outages_grouped states: ['CALIFORNIA' 'DISTRICT OF COLUMBIA' 'FLORIDA' 'LOUISIANA' 'MICHIGAN'
 'NORTH DAKOTA' 'TEXAS' 'WYOMING' 'US VIRGIN ISLANDS'
 'UNITED STATES VIRGIN ISLANDS']
df_storms_full states: ['CALIFORNIA' 'IOWA' 'KANSAS' 'TEXAS' 'VIRGINIA' 'GUAM WATERS'
 'HAWAII WATERS' 'E PACIFIC' 'GULF OF ALASKA' 'ST LAWRENCE R']


In [ ]:
print(df_outages_grouped['date'].dt.tz)
print(df_storms_full['date'].dt.tz)


None
None


In [ ]:
df_model['customers_out'].unique()

array([2.5000e+01, 2.7300e+02, 3.8700e+02, ..., 1.4490e+03, 4.9624e+04,
       5.4530e+03])

In [ ]:
# df_model.isnull().mean() * 100
# df_model.info()
# df.describe(include=['number'])
# df_model.columns
df_model[['customers_out', 'major_outage',
       'num_events', 'total_property_damage_usd', 'total_crop_damage_usd',
       'num_tornado_scales']].describe()

,customers_out,major_outage,num_events,total_property_damage_usd,total_crop_damage_usd,num_tornado_scales
count,3.033800e+04,30338.000000,30338.000000,3.033800e+04,3.033800e+04,30338.000000
mean,3.347287e+04,0.474619,1.775727,2.469232e+06,7.512939e+04,0.055112
std,4.486636e+05,0.499364,1.906335,8.849966e+07,3.280123e+06,0.258542
min,0.000000e+00,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000
25%,4.200000e+01,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000
50%,7.670000e+02,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000
75%,8.156750e+03,1.000000,2.000000,0.000000e+00,0.000000e+00,0.000000
max,3.235904e+07,1.000000,50.000000,1.000000e+10,3.080000e+08,4.000000


In [ ]:
df_model['num_dense_fog_events'].unique()

array([0., 1., 4., 3., 2., 5.])

In [ ]:
# add extra storm features (like event types: tornado, hail, flood separately
# tracks maximum wind speed and largest hail size

In [ ]:
# def prepare_merged_storm_outage_data(
#     outages_raw_df,
#     storm_raw_df,
#     state,
#     event_types,
#     start_date,
#     end_date
# ):
#     """
#     Prepare and merge outages and storm event datasets.

#     Args:
#         outages_raw_df (pd.DataFrame): Outages dataset (e.g., outages14_23_texas_df).
#         storm_raw_df (pd.DataFrame): Storm events dataset (e.g., storm_events_14_24_df).
#         state (str): State name to filter storm events (e.g., 'TEXAS').
#         event_types (list): List of event types to count (e.g., ['Hurricane', 'Flood', ...]).
#         start_date (datetime): Start date of analysis window.
#         end_date (datetime): End date of analysis window.

#     Returns:
#         merged_df (pd.DataFrame): Final merged DataFrame with features.
#         event_15min_df (pd.DataFrame): 15-min storm event time series (optional).
#     """

#     # --- Prepare Outages Dataset ---
#     outages_df = outages_raw_df.copy()
#     outages_df = outages_df.rename(columns={'run_start_time': 'Start_time'})

#     # Create Date column for grouping
#     outages_df['Date'] = outages_df['Start_time'].dt.date

#     # Group outages by fips_code and Date
#     outages_grouped_df = (
#         outages_df.groupby(['fips_code', 'Date'])
#         .agg(
#             Total_Customers_Out=('customers_out', 'sum'),
#             Number_of_Outages=('customers_out', 'count'),
#             Max_Customers_Single_Outage=('customers_out', 'max'),
#             Min_Start_Hour=('Start_time', lambda x: x.dt.hour.min()),
#             Max_Start_Hour=('Start_time', lambda x: x.dt.hour.max()),
#         )
#         .reset_index()
#     )

#     print(f"Outages grouped shape: {outages_grouped_df.shape}")

#     # --- Prepare Storm Events Dataset ---
#     storm_df = storm_raw_df.copy()

#     # Parse storm datetime columns
#     storm_df['BEGIN_DATE_TIME'] = pd.to_datetime(storm_df['BEGIN_DATE_TIME'])
#     storm_df['END_DATE_TIME'] = pd.to_datetime(storm_df['END_DATE_TIME'])

#     # Create Date columns for filtering
#     storm_df['BEGIN_DATE'] = storm_df['BEGIN_DATE_TIME'].dt.date
#     storm_df['END_DATE'] = storm_df['END_DATE_TIME'].dt.date
#     storm_df['Date'] = storm_df['BEGIN_DATE_TIME'].dt.date  # Primary Date

#     # Correct filtering logic (important!)
#     start_dt = pd.to_datetime(start_date).date()
#     end_dt = pd.to_datetime(end_date).date()

#     filtered_storm = storm_df[
#         (storm_df['STATE'] == state) &
#         (storm_df['BEGIN_DATE'] <= end_dt) &
#         (storm_df['END_DATE'] >= start_dt)
#     ].copy()

#     if filtered_storm.empty:
#         print("⚠️ Warning: No storm events after filtering. Check your date range or state name.")

#     # Additional storm features
#     filtered_storm['Event_Duration_hours'] = (filtered_storm['END_DATE_TIME'] - filtered_storm['BEGIN_DATE_TIME']).dt.total_seconds() / 3600
#     filtered_storm['Is_Overnight'] = (filtered_storm['BEGIN_DATE'] != filtered_storm['END_DATE'])
#     filtered_storm['Event_Count'] = 1

#     # Group by STATE, CZ_NAME (county), Date
#     storm_grouped_df = (
#         filtered_storm.groupby(['STATE', 'CZ_NAME', 'Date'])
#         .agg(
#             Total_Injuries=('INJURIES_DIRECT', 'sum'),
#             Total_Deaths=('DEATHS_DIRECT', 'sum'),
#             Total_Property_Damage=('DAMAGE_PROPERTY', 'sum'),
#             Mean_Event_Duration=('Event_Duration_hours', 'mean'),
#             Max_Event_Duration=('Event_Duration_hours', 'max'),
#             Proportion_Overnight_Events=('Is_Overnight', 'mean')
#         )
#         .reset_index()
#     )

#     # --- Create Event Type Daily Counts ---
#     event_type_counts = (
#         filtered_storm.pivot_table(
#             index=['STATE', 'CZ_NAME', 'Date'],
#             columns='EVENT_TYPE',
#             values='Event_Count',
#             aggfunc='sum',
#             fill_value=0
#         )
#         .reset_index()
#     )

#     # Rename event columns
#     non_event_cols = ['STATE', 'CZ_NAME', 'Date']
#     event_cols = [col for col in event_type_counts.columns if col not in non_event_cols]
#     event_type_counts.rename(columns={col: f'Event_{col}' for col in event_cols}, inplace=True)

#     # Merge grouped storm features with event type counts
#     storm_final_df = storm_grouped_df.merge(event_type_counts, on=['STATE', 'CZ_NAME', 'Date'], how='left')
#     print(f"Storm events grouped shape: {storm_final_df.shape}")

#     # Fill missing event counts with 0
#     event_count_cols = [col for col in storm_final_df.columns if col.startswith('Event_')]
#     storm_final_df[event_count_cols] = storm_final_df[event_count_cols].fillna(0)
    

#     # --- Merge Outages and Storm Events ---
#     # Merge on Date + location
#     merged_df = outages_grouped_df.merge(
#         storm_final_df,
#         left_on=['fips_code', 'Date'], 
#         right_on=['CZ_NAME', 'Date'], 
#         how='left'
#     )

#     # --- Time-Based Features (AFTER merging) ---
#     # merged_df['Date'] = pd.to_datetime(merged_df['Date'])
#     # merged_df['Year'] = merged_df['Date'].dt.year
#     # merged_df['Month'] = merged_df['Date'].dt.month
#     # merged_df['Quarter'] = merged_df['Date'].dt.quarter
#     # merged_df['Week'] = merged_df['Date'].dt.isocalendar().week
#     # merged_df['DayOfWeek'] = merged_df['Date'].dt.dayofweek

#     # --- Optional: Create 15-min storm event time series ---
#     time_index = pd.date_range(start=start_date, end=end_date, freq='15min')
#     event_15min_df = pd.DataFrame({'time': time_index})

#     # Initialize event count columns
#     for event_type in event_types:
#         event_15min_df[f'event_count_{event_type}'] = 0

#     # Round storm events to 15-min and increment counts
#     for event_type in event_types:
#         event_subset = filtered_storm[filtered_storm['EVENT_TYPE'] == event_type]

#         for _, row in event_subset.iterrows():
#             event_start = row['BEGIN_DATE_TIME'].round('15min')
#             event_end = row['END_DATE_TIME'].round('15min')

#             start_idx = event_15min_df['time'].searchsorted(event_start)
#             end_idx = event_15min_df['time'].searchsorted(event_end)

#             if start_idx < len(event_15min_df) and end_idx <= len(event_15min_df):
#                 event_15min_df.loc[start_idx:end_idx, f'event_count_{event_type}'] += 1

#     return merged_df, event_15min_df


# # Define inputs
# state = 'TEXAS'
# event_types = ['Hurricane', 'Flood', 'Thunderstorm Wind', 'Tornado', 'Winter Storm']
# start_date = datetime(2014, 1, 1)
# end_date = datetime(2023, 12, 31)

# # Call the function
# merged_df, event_15min_df = prepare_merged_storm_outage_data(
#     outages_raw_df=outages14_23_texas_df,
#     storm_raw_df=storm_events_14_24_df,
#     state=state,
#     event_types=event_types,
#     start_date=start_date,
#     end_date=end_date
# )

# merged_df

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from tqdm import tqdm


def prepare_storm_and_outages_data(
    storm_df_raw,
    outage_files_csv,
    outages_dir,
    outages_columns,
    state_filter='TEXAS',
    event_types=None,
    start_date='2014-01-01',
    end_date='2023-12-31'
):
    if event_types is None:
        event_types = ['Tropical Storm', 'Hurricane', 'Flood', 'Thunderstorm Wind', 'High Wind', 'Heavy Rain']

    # Convert date strings to datetime
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date) + pd.Timedelta(hours=23, minutes=45)

    # --- OUTAGES PREPARATION ---
    filtered_outages = []
    for f in tqdm(outage_files_csv, desc=f"Reading outages for {state_filter}"):
        file_path = os.path.join(outages_dir, f)
        df = pd.read_csv(file_path, usecols=lambda c: c in outages_columns)
        df_state = df[df['state'].str.upper() == state_filter.upper()]
        filtered_outages.append(df_state)

    outages_df = pd.concat(filtered_outages, ignore_index=True)
    outages_df = outages_df.rename(columns={'run_start_time': 'Start_time'})
    outages_df['Start_time'] = pd.to_datetime(outages_df['Start_time'])

    outages_df['Start_Hour'] = outages_df['Start_time'].dt.hour
    outages_df['Start_DayofWeek'] = outages_df['Start_time'].dt.dayofweek
    outages_df['Start_Month'] = outages_df['Start_time'].dt.month
    outages_df['Start_Quarter'] = outages_df['Start_time'].dt.quarter
    outages_df['Date'] = outages_df['Start_time'].dt.date

    outages_grouped_df = (
        outages_df.groupby(['fips_code', 'Date'])
        .agg(
            Total_Customers_Out=('customers_out', 'sum'),
            Number_of_Outages=('customers_out', 'count'),
            Max_Customers_Single_Outage=('customers_out', 'max'),
            Min_Start_Hour=('Start_Hour', 'min'),
            Max_Start_Hour=('Start_Hour', 'max'),
            Start_DayofWeek=('Start_DayofWeek', 'first'),
            Start_Month=('Start_Month', 'first'),
            Start_Quarter=('Start_Quarter', 'first')
        )
        .reset_index()
    )

    # --- STORM PREPARATION ---
    storm_df = storm_df_raw.copy()
    storm_df['BEGIN_DATE_TIME'] = pd.to_datetime(storm_df['BEGIN_DATE_TIME'])
    storm_df['END_DATE_TIME'] = pd.to_datetime(storm_df['END_DATE_TIME'])

    storm_df = storm_df[
        (storm_df['STATE'].str.upper() == state_filter.upper()) &
        (storm_df['END_DATE_TIME'] >= start_dt) &
        (storm_df['BEGIN_DATE_TIME'] <= end_dt) &
        (storm_df['EVENT_TYPE'].isin(event_types))
    ]

    storm_df['Start_Hour'] = storm_df['BEGIN_DATE_TIME'].dt.hour
    storm_df['End_Hour'] = storm_df['END_DATE_TIME'].dt.hour
    storm_df['Start_DayofWeek'] = storm_df['BEGIN_DATE_TIME'].dt.dayofweek
    storm_df['Start_Month'] = storm_df['BEGIN_DATE_TIME'].dt.month
    storm_df['Start_Quarter'] = storm_df['BEGIN_DATE_TIME'].dt.quarter
    storm_df['Event_Duration_hours'] = (storm_df['END_DATE_TIME'] - storm_df['BEGIN_DATE_TIME']).dt.total_seconds() / 3600
    storm_df['Is_Overnight'] = (storm_df['BEGIN_DATE_TIME'].dt.date != storm_df['END_DATE_TIME'].dt.date)
    storm_df['Date'] = storm_df['BEGIN_DATE_TIME'].dt.date
    storm_df['Event_Count'] = 1
    storm_df['DAMAGE_PROPERTY'] = storm_df['DAMAGE_PROPERTY'].apply(parse_damage)

    # Event type pivot
    event_type_counts = (
        storm_df.pivot_table(
            index=['STATE', 'CZ_NAME', 'Date'],
            columns='EVENT_TYPE',
            values='Event_Count',
            aggfunc='sum',
            fill_value=0
        )
        .reset_index()
    )

    # Rename event columns
    non_event_cols = ['STATE', 'CZ_NAME', 'Date']
    event_cols = [col for col in event_type_counts.columns if col not in non_event_cols]
    event_type_counts.rename(columns={col: f'Event_{col}' for col in event_cols}, inplace=True)

    # Group storm features
    storm_grouped_df = (
        storm_df.groupby(['STATE', 'CZ_NAME', 'Date'])
        .agg(
            Total_Injuries=('INJURIES_DIRECT', 'sum'),
            Total_Deaths=('DEATHS_DIRECT', 'sum'),
            Total_Property_Damage=('DAMAGE_PROPERTY', 'sum'),
            Mean_Event_Duration=('Event_Duration_hours', 'mean'),
            Max_Event_Duration=('Event_Duration_hours', 'max'),
            Proportion_Overnight_Events=('Is_Overnight', 'mean'),
            Start_DayofWeek=('Start_DayofWeek', 'first'),
            Start_Month=('Start_Month', 'first'),
            Start_Quarter=('Start_Quarter', 'first')
        )
        .reset_index()
    )

    storm_final_df = storm_grouped_df.merge(event_type_counts, on=['STATE', 'CZ_NAME', 'Date'], how='left')
    for col in [c for c in storm_final_df.columns if c.startswith('Event_')]:
        storm_final_df[col] = storm_final_df[col].fillna(0)

    return outages_grouped_df, storm_final_df


outages_df, storm_df = prepare_storm_and_outages_data(
    storm_df_raw=storm_events_14_24_df,
    outage_files_csv=outage_files_csv,
    outages_dir=outages_dir,
    outages_columns=outages.columns,
    state_filter='Florida',
    start_date='2014-01-01',
    end_date='2023-12-31'
)


Reading outages for Florida:  60%|██████    | 6/10 [00:48<00:42, 10.52s/it]

: 

In [ ]:
import pandas as pd
from datetime import datetime

def prepare_storm_and_outage_data(
    storm_events_df,
    outages_df,
    state_filter="FLORIDA",
    event_types=None,
    start_date=datetime(2014, 1, 1),
    end_date=datetime(2023, 12, 31, 23, 45),
    parse_damage_func=None
):
    if event_types is None:
        event_types = ['Tropical Storm', 'Hurricane', 'Flood', 'Thunderstorm Wind', 'High Wind', 'Heavy Rain']

    # --- Prepare Outages Dataset ---
    outages_df = outages_df.copy()
    outages_df = outages_df.rename(columns={'run_start_time': 'Start_time'})

    outages_df['Start_Hour'] = outages_df['Start_time'].dt.hour
    outages_df['Start_DayofWeek'] = outages_df['Start_time'].dt.dayofweek
    outages_df['Start_Month'] = outages_df['Start_time'].dt.month
    outages_df['Start_Quarter'] = outages_df['Start_time'].dt.quarter
    outages_df['Date'] = outages_df['Start_time'].dt.date

    outages_grouped_df = (
        outages_df.groupby(['fips_code', 'Date'])
        .agg(
            Total_Customers_Out=('customers_out', 'sum'),
            Number_of_Outages=('customers_out', 'count'),
            Max_Customers_Single_Outage=('customers_out', 'max'),
            Min_Start_Hour=('Start_Hour', 'min'),
            Max_Start_Hour=('Start_Hour', 'max'),
            Start_DayofWeek=('Start_DayofWeek', 'first'),
            Start_Month=('Start_Month', 'first'),
            Start_Quarter=('Start_Quarter', 'first')
        )
        .reset_index()
    )

    # --- Prepare Storm Events Dataset ---
    storm_df = storm_events_df.copy()
    storm_df['BEGIN_DATE_TIME'] = pd.to_datetime(storm_df['BEGIN_DATE_TIME'])
    storm_df['END_DATE_TIME'] = pd.to_datetime(storm_df['END_DATE_TIME'])

    storm_df['Start_Hour'] = storm_df['BEGIN_DATE_TIME'].dt.hour
    storm_df['End_Hour'] = storm_df['END_DATE_TIME'].dt.hour
    storm_df['Start_DayofWeek'] = storm_df['BEGIN_DATE_TIME'].dt.dayofweek
    storm_df['Start_Month'] = storm_df['BEGIN_DATE_TIME'].dt.month
    storm_df['Start_Quarter'] = storm_df['BEGIN_DATE_TIME'].dt.quarter
    storm_df['Event_Duration_hours'] = (storm_df['END_DATE_TIME'] - storm_df['BEGIN_DATE_TIME']).dt.total_seconds() / 3600
    storm_df['Is_Overnight'] = (storm_df['BEGIN_DATE_TIME'].dt.date != storm_df['END_DATE_TIME'].dt.date)
    storm_df['Date'] = storm_df['BEGIN_DATE_TIME'].dt.date
    storm_df['Event_Count'] = 1

    # Apply damage parser if provided
    if parse_damage_func:
        storm_df['DAMAGE_PROPERTY'] = storm_df['DAMAGE_PROPERTY'].apply(parse_damage_func)

    # Pivot: Daily event type counts
    event_type_counts = (
        storm_df.pivot_table(
            index=['STATE', 'CZ_NAME', 'Date'],
            columns='EVENT_TYPE',
            values='Event_Count',
            aggfunc='sum',
            fill_value=0
        )
        .reset_index()
    )

    non_event_cols = ['STATE', 'CZ_NAME', 'Date']
    event_cols = [col for col in event_type_counts.columns if col not in non_event_cols]
    event_type_counts.rename(columns={col: f'Event_{col}' for col in event_cols}, inplace=True)

    # Group: Daily storm stats
    storm_grouped_df = (
        storm_df.groupby(['STATE', 'CZ_NAME', 'Date'])
        .agg(
            Total_Injuries=('INJURIES_DIRECT', 'sum'),
            Total_Deaths=('DEATHS_DIRECT', 'sum'),
            Total_Property_Damage=('DAMAGE_PROPERTY', 'sum'),
            Mean_Event_Duration=('Event_Duration_hours', 'mean'),
            Max_Event_Duration=('Event_Duration_hours', 'max'),
            Proportion_Overnight_Events=('Is_Overnight', 'mean'),
            Start_DayofWeek=('Start_DayofWeek', 'first'),
            Start_Month=('Start_Month', 'first'),
            Start_Quarter=('Start_Quarter', 'first')
        )
        .reset_index()
    )

    storm_final_df = storm_grouped_df.merge(event_type_counts, on=['STATE', 'CZ_NAME', 'Date'], how='left')

    # Fill missing event counts with 0
    event_count_cols = [col for col in storm_final_df.columns if col.startswith('Event_')]
    storm_final_df[event_count_cols] = storm_final_df[event_count_cols].fillna(0)

    return storm_final_df, outages_grouped_df


storm_final_df, outages_grouped_df = prepare_storm_and_outage_data(
    storm_events_df=storm_events_14_24_df,
    outages_df=outages14_23_texas_df,
    state_filter="FLORIDA",
    parse_damage_func=parse_damage
)
storm_final_df

In [ ]:
import pandas as pd
from datetime import datetime



def prepare_merged_storm_outage_data(storm_df, outages_df, state='FLORIDA', event_types=None,
                                     start_date='2014-01-01', end_date='2023-12-31'):
    # --- Prepare outages ---
    outages_df = outages_df.copy()
    outages_df = outages_df.rename(columns={'run_start_time': 'Start_time'})
    outages_df['Date'] = outages_df['Start_time'].dt.date
    outages_df['Start_Hour'] = outages_df['Start_time'].dt.hour
    outages_df['Start_DayofWeek'] = outages_df['Start_time'].dt.dayofweek
    outages_df['Start_Month'] = outages_df['Start_time'].dt.month
    outages_df['Start_Quarter'] = outages_df['Start_time'].dt.quarter

    outages_grouped = (
        outages_df.groupby(['fips_code', 'Date'])
        .agg(
            Total_Customers_Out=('customers_out', 'sum'),
            Number_of_Outages=('customers_out', 'count'),
            Max_Customers_Single_Outage=('customers_out', 'max'),
            Min_Start_Hour=('Start_Hour', 'min'),
            Max_Start_Hour=('Start_Hour', 'max'),
            Start_DayofWeek=('Start_DayofWeek', 'first'),
            Start_Month=('Start_Month', 'first'),
            Start_Quarter=('Start_Quarter', 'first')
        )
        .reset_index()
    )

    # --- Prepare storm events ---
    storm_df = storm_df.copy()
    storm_df['BEGIN_DATE_TIME'] = pd.to_datetime(storm_df['BEGIN_DATE_TIME'])
    storm_df['END_DATE_TIME'] = pd.to_datetime(storm_df['END_DATE_TIME'])

    storm_df['Start_Hour'] = storm_df['BEGIN_DATE_TIME'].dt.hour
    storm_df['End_Hour'] = storm_df['END_DATE_TIME'].dt.hour
    storm_df['Start_DayofWeek'] = storm_df['BEGIN_DATE_TIME'].dt.dayofweek
    storm_df['Start_Month'] = storm_df['BEGIN_DATE_TIME'].dt.month
    storm_df['Start_Quarter'] = storm_df['BEGIN_DATE_TIME'].dt.quarter
    storm_df['Event_Duration_hours'] = (storm_df['END_DATE_TIME'] - storm_df['BEGIN_DATE_TIME']).dt.total_seconds() / 3600
    storm_df['Is_Overnight'] = storm_df['BEGIN_DATE_TIME'].dt.date != storm_df['END_DATE_TIME'].dt.date
    storm_df['Date'] = storm_df['BEGIN_DATE_TIME'].dt.date
    storm_df['Event_Count'] = 1
    storm_df['DAMAGE_PROPERTY'] = storm_df['DAMAGE_PROPERTY'].apply(parse_damage)

    # Filter storm data by state and date range
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    filtered_storm = storm_df[
        (storm_df['STATE'] == state) &
        (storm_df['BEGIN_DATE_TIME'] <= end_dt) &
        (storm_df['END_DATE_TIME'] >= start_dt)
    ]

    # Pivot for event type counts
    event_counts = (
        filtered_storm.pivot_table(
            index=['STATE', 'CZ_NAME', 'Date'],
            columns='EVENT_TYPE',
            values='Event_Count',
            aggfunc='sum',
            fill_value=0
        )
        .reset_index()
    )
    non_event_cols = ['STATE', 'CZ_NAME', 'Date']
    event_cols = [col for col in event_counts.columns if col not in non_event_cols]
    event_counts.rename(columns={col: f'Event_{col}' for col in event_cols}, inplace=True)

    # Aggregate storm stats
    storm_grouped = (
        filtered_storm.groupby(['STATE', 'CZ_NAME', 'Date'])
        .agg(
            Total_Injuries=('INJURIES_DIRECT', 'sum'),
            Total_Deaths=('DEATHS_DIRECT', 'sum'),
            Total_Property_Damage=('DAMAGE_PROPERTY', 'sum'),
            Mean_Event_Duration=('Event_Duration_hours', 'mean'),
            Max_Event_Duration=('Event_Duration_hours', 'max'),
            Proportion_Overnight_Events=('Is_Overnight', 'mean'),
            Start_DayofWeek=('Start_DayofWeek', 'first'),
            Start_Month=('Start_Month', 'first'),
            Start_Quarter=('Start_Quarter', 'first')
        )
        .reset_index()
    )

    storm_final = storm_grouped.merge(event_counts, on=['STATE', 'CZ_NAME', 'Date'], how='left')
    storm_final.update(storm_final.select_dtypes(include='number').fillna(0))

    print(f"Outages grouped shape: {outages_grouped.shape}")
    print(f"Storm events grouped shape: {storm_final.shape}")

    return storm_final, outages_grouped


storm_final_df, outages_grouped_df = prepare_merged_storm_outage_data(
    storm_events_14_24_df,
    outages14_23_texas_df,
    state='FLORIDA',
    event_types=['Tropical Storm', 'Hurricane', 'Flood', 'Thunderstorm Wind', 'High Wind', 'Heavy Rain']
)


/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_87278/3164795041.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  storm_df['END_DATE_TIME'] = pd.to_datetime(storm_df['END_DATE_TIME'])


Outages grouped shape: (522485, 10)
Storm events grouped shape: (0, 12)


In [ ]:
outages_grouped_df['Total_Customers_Out'].unique()

array([2.50000e+01, 7.70000e+01, 1.37000e+02, ..., 1.17735e+05,
       4.23060e+04, 1.53840e+04])

In [ ]:
# --- Step 1: Prepare Outages Dataset ---

outages_df = outages14_23_texas_df.copy()

outages_df = outages_df.rename(columns={'run_start_time': 'Start_time'})

outages_df['Start_Hour'] = outages_df['Start_time'].dt.hour
outages_df['Start_DayofWeek'] = outages_df['Start_time'].dt.dayofweek
outages_df['Start_Month'] = outages_df['Start_time'].dt.month
outages_df['Start_Quarter'] = outages_df['Start_time'].dt.quarter
outages_df['Date'] = outages_df['Start_time'].dt.date

outages_grouped_df = (
    outages_df.groupby(['fips_code', 'Date'])
    .agg(
        Total_Customers_Out=('customers_out', 'sum'),
        Number_of_Outages=('customers_out', 'count'),
        Max_Customers_Single_Outage=('customers_out', 'max'),
        Min_Start_Hour=('Start_Hour', 'min'),
        Max_Start_Hour=('Start_Hour', 'max'),
        Start_DayofWeek=('Start_DayofWeek', 'first'),
        Start_Month=('Start_Month', 'first'),
        Start_Quarter=('Start_Quarter', 'first')
    )
    .reset_index()
)

print(f"Outages grouped shape: {outages_grouped_df.shape}")


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# --- Step 2: Prepare Storm Events Dataset ---

storm_df = storm_events_14_24_df.copy()

# Convert datetime columns
storm_df['BEGIN_DATE_TIME'] = pd.to_datetime(storm_df['BEGIN_DATE_TIME'])
storm_df['END_DATE_TIME'] = pd.to_datetime(storm_df['END_DATE_TIME'])

# Extract time-based features
storm_df['Start_Hour'] = storm_df['BEGIN_DATE_TIME'].dt.hour
storm_df['End_Hour'] = storm_df['END_DATE_TIME'].dt.hour
storm_df['Start_DayofWeek'] = storm_df['BEGIN_DATE_TIME'].dt.dayofweek
storm_df['Start_Month'] = storm_df['BEGIN_DATE_TIME'].dt.month
storm_df['Start_Quarter'] = storm_df['BEGIN_DATE_TIME'].dt.quarter
storm_df['Event_Duration_hours'] = (storm_df['END_DATE_TIME'] - storm_df['BEGIN_DATE_TIME']).dt.total_seconds() / 3600
storm_df['Is_Overnight'] = (storm_df['BEGIN_DATE_TIME'].dt.date != storm_df['END_DATE_TIME'].dt.date)
storm_df['Date'] = storm_df['BEGIN_DATE_TIME'].dt.date
storm_df['Event_Count'] = 1

# --- (Optional) 15-Minute Event Count Time Series ---

# Define dynamically based on your analysis window and filtering needs
start_year, start_month, start_day = 2014, 1, 1
end_year, end_month, end_day = 2023, 12, 31

state = "FLORIDA"  # or dynamically from site metadata
event_types = ['Tropical Storm', 'Hurricane', 'Flood', 'Thunderstorm Wind', 'High Wind', 'Heavy Rain']


start_date = datetime(start_year, start_month, start_day)
end_date = datetime(end_year, end_month, end_day, 23, 45)
time_index = pd.date_range(start=start_date, end=end_date, freq='15min')
event_15min_df = pd.DataFrame({'time': time_index})

# Initialize event count columns
for event_type in event_types:
    event_15min_df[f'event_count {event_type}'] = 0

# Round storm events to 15-min intervals and increment counts
filtered_df = storm_df[
    (storm_df['STATE'] == state) &
    (storm_df['EVENT_TYPE'].isin(event_types)) &
    (storm_df['END_DATE_TIME'] >= start_date) &
    (storm_df['BEGIN_DATE_TIME'] <= end_date)
].copy()

for event_type in event_types:
    event_subset = filtered_df[filtered_df['EVENT_TYPE'] == event_type]

    for _, row in event_subset.iterrows():
        event_start = row['BEGIN_DATE_TIME'].round('15min')
        event_end = row['END_DATE_TIME'].round('15min')

        start_idx = event_15min_df['time'].searchsorted(event_start)
        end_idx = event_15min_df['time'].searchsorted(event_end)

        if start_idx < len(event_15min_df) and end_idx <= len(event_15min_df):
            event_15min_df.loc[start_idx:end_idx, f'event_count {event_type}'] += 1

# --- Event Type Daily Count Pivot (for spatial join) ---

event_type_counts = (
    storm_df.pivot_table(
        index=['STATE', 'CZ_NAME', 'Date'],
        columns='EVENT_TYPE',
        values='Event_Count',
        aggfunc='sum',
        fill_value=0
    )
    .reset_index()
)

# Rename event columns
non_event_cols = ['STATE', 'CZ_NAME', 'Date']
event_cols = [col for col in event_type_counts.columns if col not in non_event_cols]
event_type_counts.rename(columns={col: f'Event_{col}' for col in event_cols}, inplace=True)

# --- Apply damage parser (must define parse_damage beforehand) ---
storm_df['DAMAGE_PROPERTY'] = storm_df['DAMAGE_PROPERTY'].apply(parse_damage)

# --- Aggregate storm-level features ---
storm_grouped_df = (
    storm_df.groupby(['STATE', 'CZ_NAME', 'Date'])
    .agg(
        Total_Injuries=('INJURIES_DIRECT', 'sum'),
        Total_Deaths=('DEATHS_DIRECT', 'sum'),
        Total_Property_Damage=('DAMAGE_PROPERTY', 'sum'),
        Mean_Event_Duration=('Event_Duration_hours', 'mean'),
        Max_Event_Duration=('Event_Duration_hours', 'max'),
        Proportion_Overnight_Events=('Is_Overnight', 'mean'),
        Start_DayofWeek=('Start_DayofWeek', 'first'),
        Start_Month=('Start_Month', 'first'),
        Start_Quarter=('Start_Quarter', 'first')
    )
    .reset_index()
)

# --- Final merge with event type counts ---
storm_final_df = storm_grouped_df.merge(event_type_counts, on=['STATE', 'CZ_NAME', 'Date'], how='left')

# Fill missing event counts with 0
event_count_cols = [col for col in storm_final_df.columns if col.startswith('Event_')]
storm_final_df[event_count_cols] = storm_final_df[event_count_cols].fillna(0)

print(f"Storm events grouped shape: {storm_final_df.shape}")


/var/folders/fz/x4myp3c97fx7r5kdg1c9gzmc0000gn/T/ipykernel_84689/3653128900.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  storm_df['END_DATE_TIME'] = pd.to_datetime(storm_df['END_DATE_TIME'])


Storm events grouped shape: (33685, 50)


In [ ]:
storm_final_df['Event_Winter Weather'].unique()
# storm_final_df['Total_Injuries'].unique()

array([0, 1, 3, 2, 4])

In [ ]:



# # First, pivot event types before aggregation
# event_type_counts = (
#     df.pivot_table(
#         index=['StationID', 'Date'],
#         columns='Event_Type',
#         values='Customers_Affected',  # or any column (use 'size' later)
#         aggfunc='count',
#         fill_value=0
#     )
#     .reset_index()
# )

# # Rename columns: make them easier to work with
# event_type_counts.columns = [f'EventType_{col}' if not isinstance(col, tuple) else f'EventType_{col[1]}' for col in event_type_counts.columns]

# # Now merge with the original grouped dataframe
# grouped_df = (
#     df.groupby(['StationID', 'Date'])
#     .agg(
#         Total_Customers_Affected=('Customers_Affected', 'sum'),
#         Number_of_Events=('Customers_Affected', 'count'),
#         Mean_Event_Duration=('Event_Duration_hours', 'mean'),
#         Max_Customers_Single_Event=('Customers_Affected', 'max'),
#         Min_Start_Hour=('Start_Hour', 'min'),
#         Max_End_Hour=('End_Hour', 'max'),
#         Proportion_Overnight_Events=('Is_Overnight', 'mean'),
#         Start_DayofWeek=('Start_DayofWeek', 'first'),
#         Start_Month=('Start_Month', 'first'),
#         Start_Quarter=('Start_Quarter', 'first')
#     )
#     .reset_index()
# )

# # Final merge: add the event type counts
# grouped_df = grouped_df.merge(event_type_counts, on=['StationID', 'Date'], how='left')


In [ ]:
storm_final_df['STATE'].unique()
# storm_grouped_df['Total_Property_Damage'].describe()
# event_type_counts['Event_Drought'].describe()


array(['TEXAS'], dtype=object)

In [ ]:
# # Step 1: Get combined daily data
# df_hourly, df_daily = combine_agg_ts(state="Florida", 
#                                      start_year=2014, start_month=1, start_day=1, 
#                                      end_year=2023, end_month=12, end_day=31)


In [ ]:
# ggggg

In [ ]:

# Choose df_daily for now
df_final = df_daily.copy()

# Step 2: Add time-based features
df_final['day_of_week'] = df_final.index.dayofweek  # 0 = Monday
df_final['is_weekend'] = df_final['day_of_week'].isin([5, 6]).astype(int)

# Step 3: Add lag features
df_final['customers_out_lag1'] = df_final['customers_out'].shift(1)
df_final['customers_out_lag7'] = df_final['customers_out'].shift(7)

# Step 4: Add rolling window features
df_final['customers_out_roll3'] = df_final['customers_out'].rolling(window=3, min_periods=1).mean()
df_final['customers_out_roll7'] = df_final['customers_out'].rolling(window=7, min_periods=1).mean()

# Step 5: Fill any remaining NaNs if needed
df_final.fillna(0, inplace=True)

# Step 6 (Optional): Scale customers_out if needed (e.g., MinMaxScaler, StandardScaler)
# Only necessary if you plan to use linear models or neural nets

# Now df_final is model-ready!


: 

In [ ]:
fsds